# BAIXANDO OS DADOS DO GEOFABRICK

In [ ]:
import os
os.chdir("../")

from modules.geofabrik import ProtobufDownloader

PBD = ProtobufDownloader()
PBD.run()

# TRANSFORMANDO PBF PARA O5M PARA REALIZAR FILTROS E DIMINUIR TAMANHO DO PROTOBUF

In [ ]:
import os
os.chdir("../")

from modules.osmtools.osm_convert import OSMConvert

OSMC = OSMConvert(
    type_osm_in='pbf',
    type_osm_out='o5m',
)

OSMC.input_file             = 'brazil-latest.osm.pbf'
OSMC.drop_author            = True
OSMC.drop_version           = True
OSMC.verbose                = True
OSMC.complete_ways          = True
OSMC.complete_multipolygons = True
OSMC.max_objects            = 500000000
OSMC.hash_memory            = 4096

OSMC.run()

# REALIZANDO FILTRAGEM DE DADOS NO PROTOBUF

In [ ]:
# import os
# os.chdir("../")

from modules.osmtools.osm_filter import OSMfilter

OSMF = OSMfilter(verbose=True)
OSMF.input_file = 'brazil-latest.osm.o5m'

OSMF.run()

# CONVERTENDO O5M PARA PROTOBUF

In [ ]:
import os
os.chdir("../")

from modules.osmtools.osm_convert import OSMConvert

OSMC = OSMConvert(
    base_path_in = os.path.join("data","processed"),
    base_path_out = os.path.join("data","processed"),
    type_osm_in='o5m',
    type_osm_out='pbf',
)

OSMC.input_file             = 'brazil-latest.osm.filtered.streets.o5m'
OSMC.drop_author            = False
OSMC.drop_version           = False
OSMC.verbose                = False
OSMC.complete_ways          = False
OSMC.complete_multipolygons = False
OSMC.max_objects            = 500000000
OSMC.hash_memory            = 4096

OSMC.run()

# CRIANDO O BANCO COM RODOVIAS E SEUS LINKS COM O PROTOBUF FILTRADO

In [1]:
import os
os.chdir("../")

In [2]:
from modules.osmtools.spatialite import *

SP = SpatialiteOsmNet()
# SP.help()

args = [
    "-o",
    os.path.join("data","processed","pbf","brazil-latest.osm.filtered.streets.pbf"),
    "-T",
    "roads",
    "-d",
    "streets.sqlite",
    # "--roads"
]
SP.run(args=args)

Running command: modules\osmtools\bin\Windows\spatialite\tools\spatialite_osm_net.exe -o data\processed\pbf\brazil-latest.osm.filtered.streets.pbf -T roads -d streets.sqlite
stdout: SQLite version: 3.33.0
SpatiaLite version: 5.0.0


Parsing input: Pass 1 [Nodes and Ways] ...
Parsing input: Pass 2 [Arcs of the Graph] ...

Creating helper table 'roads_nodes' ... wait please ...
	Helper table 'roads_nodes' successfully created

Dropping temporary table 'osm_tmp_nodes' ... wait please ...
	Dropped table 'osm_tmp_nodes'

Dropping temporary table 'graph_nodes' ... wait please ...
	Dropped table 'graph_nodes'

VACUUMing the DB ... wait please ...
	All done: OSM graph was successfully loaded



CompletedProcess(args=['modules\\osmtools\\bin\\Windows\\spatialite\\tools\\spatialite_osm_net.exe', '-o', 'data\\processed\\pbf\\brazil-latest.osm.filtered.streets.pbf', '-T', 'roads', '-d', 'streets.sqlite'], returncode=0, stdout="SQLite version: 3.33.0\nSpatiaLite version: 5.0.0\n\n\nParsing input: Pass 1 [Nodes and Ways] ...\nParsing input: Pass 2 [Arcs of the Graph] ...\n\nCreating helper table 'roads_nodes' ... wait please ...\n\tHelper table 'roads_nodes' successfully created\n\nDropping temporary table 'osm_tmp_nodes' ... wait please ...\n\tDropped table 'osm_tmp_nodes'\n\nDropping temporary table 'graph_nodes' ... wait please ...\n\tDropped table 'graph_nodes'\n\nVACUUMing the DB ... wait please ...\n\tAll done: OSM graph was successfully loaded\n", stderr='')

# CRIANDO A TABELA DE ROTEIRIZAÇÃO

In [6]:
from modules.osmtools.spatialite import *

SP = SpatialiteNetwork()
# SP.help()

args = [
    "-d",
    "./streets.sqlite",
    "-T",
    "roads",
    "-f",
    "node_from",
    "-t",
    "node_to",
    "-g",
    "geometry",
    "-c",
    "cost",
    "--a-star-supported",
    "-n",
    "name",
    "-o",
    "table_router_time",
    "-vt",
    "router_time",
    "--overwrite-output"
]
SP.run(args=args)

Running command: modules\osmtools\bin\Windows\spatialite\tools\spatialite_network.exe -d ./streets.sqlite -T roads -f node_from -t node_to -g geometry -c cost --a-star-supported -n name -o table_router_time -vt router_time --overwrite-output
stdout: 
spatialite-network

   SpatiaLite db: ./streets.sqlite
validating table: roads

columns layout
FromNode: node_from
  ToNode: node_to
    Cost: cost
    Name: name
Geometry: geometry

assuming arcs to be BIDIRECTIONAL

NETWORK-DATA table creation required: 'table_router_time'

VirtualNetwork table creation required: 'router_time'
Overwrite allowed if table already exists


Statistics
	# Arcs : 24018250
	# Nodes: 9405115
	Node max  incoming arcs: 8
	Node max outcoming arcs: 8
	# Nodes   cardinality=1: 1878571 [terminal nodes]
	# Nodes   cardinality=2: 972595 [meaningless, pass-through]


OK: network passed validation
	you can apply this configuration to build a valid VirtualNetwork


OK: NETWORK-DATA table 'table_router_time' successfully cr

CompletedProcess(args=['modules\\osmtools\\bin\\Windows\\spatialite\\tools\\spatialite_network.exe', '-d', './streets.sqlite', '-T', 'roads', '-f', 'node_from', '-t', 'node_to', '-g', 'geometry', '-c', 'cost', '--a-star-supported', '-n', 'name', '-o', 'table_router_time', '-vt', 'router_time', '--overwrite-output'], returncode=0, stdout="\nspatialite-network\n\n==================================================================\n   SpatiaLite db: ./streets.sqlite\nvalidating table: roads\n\ncolumns layout\n==================================================================\nFromNode: node_from\n  ToNode: node_to\n    Cost: cost\n    Name: name\nGeometry: geometry\n\nassuming arcs to be BIDIRECTIONAL\n\nNETWORK-DATA table creation required: 'table_router_time'\n\nVirtualNetwork table creation required: 'router_time'\nOverwrite allowed if table already exists\n==================================================================\n\n\nStatistics\n===============================================

geral

In [ ]:
all_names = """
   10006695     building
    7710843     highway
    6257706     source
    4055046     name
    3578020     height
    2725036     surface
    1690067     natural
    1634041     addr:street
    1491832     waterway
    1450681     addr:housenumber
    1380072     building:levels
    1273811     power
    1247236     oneway
     870462     addr:city
     807491     pmfsefin:idedif
     804945     addr:suburb
     720460     addr:postcode
     694912     landuse
     554155     lanes
     486215     maxspeed
     452810     layer
     438206     amenity
     417098     intermittent
     394014     access
     380846     building:part
     369588     ref
     362293     barrier
     348842     type
     345219     leaf_type
     322778     alt_name
     308581     leisure
     301819     leaf_cycle
     289909     service
     289725     generator:source
     287513     generator:method
     278670     generator:type
     276757     ele
     266283     place
     255094     generator:output:electricity
     231408     lit
     215031     tunnel
     195447     crossing
     178931     operator
     177978     description
     170200     water
     163032     shop
     154500     source:name
     146844     man_made
     145513     public_transport
     142666     bridge
     128066     bus
     120480     restriction
      99261     postal_code
      90776     noexit
      90660     traffic_calming
      89226     footway
      86105     sport
      85705     roof:shape
      83504     bicycle
      82873     note
      78256     foot
      73553     trees
      70237     phone
      69729     tracktype
      69445     crossing:markings
      68212     source:date
      67725     sidewalk
      67652     horse
      66283     denotation
      60445     roof:colour
      53867     created_by
      52433     old_name
      52371     IBGE:CD_ADMINIS
      51732     railway
      50378     opening_hours
      49897     junction
      48991     religion
      48303     wheelchair
      48226     boundary
      45907     material
      44507     brand
      44253     area
      43614     top_ele
      43604     wetland
      43374     network
      42734     building:material
      41932     IPP:CodEdificio
      41723     website
      41598     crossing_ref
      41587     location
      41381     admin_level
      40815     official_name
      40392     produce
      40246     tower:type
      39245     roof:material
      38470     level
      37774     brand:wikidata
      37681     genus
      37510     smoothness
      37023     ford
      36689     width
      36237     shelter
      36168     source:highway
      36137     tourism
      36031     direction
      34763     wikipedia
      34325     healthcare
      34207     operator:type
      32837     building:use
      31869     wikidata
      31206     crop
      30470     communication:mobile_phone
      29582     denomination
      28580     ref:cadastre
      28474     seasonal
      27862     addr:country
      27727     office
      27683     motor_vehicle
      27497     taxon
      27457     taxon:pt
      27182     generator:solar:tracking
      26901     fixme
      26396     source:maxspeed
      25715     bench
      25097     covered
      24761     aeroway
      23558     parking
      23530     tactile_paving
      23514     destination
      23434     entrance
      23378     short_name
      23247     source:geometry
      22962     cuisine
      22119     residential
      22058     landcover
      21741     traffic_signals
      20165     gauge
      19062     source:highway_classification
      18632     route_ref
      18628     electrified
      18520     building:colour
      18513     noname
      18227     voltage
      18176     frequency
      17996     fee
      17836     brand:wikipedia
      17753     operator:wikidata
      17488     population
      17288     gtfs_id
      17118     addr:state
      16889     crossing:island
      16682     IBGE:GEOCODIGO
      16394     contact:phone
      16346     kerb
      16088     design
      16024     cables
      15605     FITOFISION
      15221     source:postal_code
      15130     cycleway:right
      15027     source:population
      14900     email
      14864     wires
      14797     circuits
      14537     start_date
      14327     roof:orientation
      14132     addr:interpolation
      14012     loc_name
      13490     ref:AGESIC
      13397     is_in:state
      13153     capacity
      13001     name:en
      12936     usage
      12918     route
      12911     departures_board
      12660     fence_type
      12610     network:wikidata
      12554     lane_markings
      12506     product
      12341     addr:inclusion
      12272     maxspeed:hgv
      12226     cycleway:left
      12165     shoulder
      12145     addr:place
      12116     house
      11981     check_date
      11852     owner
      11783     colour
      11750     reservation
      11680     historic
      11676     is_in:municipality
      11569     population:date
      11456     roof:height
      11372     distance
      11116     name:pt
      10889     emergency
      10830     indoor
      10792     seamark:type
      10388     turn:lanes
      10217     construction
      10187     destination:ref
       9854     oneway:bicycle
       9769     ownership
       9666     IBGE:CD_CLASSE_
       9584     survey:date
       9395     internet_access
       9115     narrow
       9074     addr:unit
       9002     payment:cash
       8976     old_ref
       8938     aquaculture
       8915     structure
       8682     cycleway
       8640     payment:credit_cards
       8523     public_transport:version
       8502     incline
       8174     content
       7857     bin
       7734     lanes:forward
       7610     lanes:backward
       7554     payment:debit_cards
       7464     substation
       7386     supervised
       7274     traffic_sign
       7103     species
       6798     crown_type
       6767     smoking
       6759     IBGE:COD_UNICO_ENDERECO
       6707     from
       6707     to
       6656     industrial
       6577     roof:levels
       6565     addr:district
       6155     craft
       6131     addr:housename
       5977     backrest
       5931     loc_ref
       5916     source:ref
       5893     local_ref
       5869     stop
       5738     maxheight
       5650     dispensing
       5638     operator:wikipedia
       5623     boat
       5610     wall
       5584     atm
       5562     branch
       5550     government
       5534     healthcare:speciality
       5447     network:wikipedia
       5374     toll
       5171     segregated
       5158     traffic_signals:direction
       5152     nohousenumber
       5114     source:addr
       5076     air_conditioning
       5072     dog
       4971     lamp_type
       4896     takeaway
       4633     informal
       4617     contact:website
       4444     hgv
       4340     vehicle
       4280     min_height
       4249     maxspeed:type
       4215     sidewalk:right
       4207     drive_through
       4195     motorcar
       4059     sidewalk:left
       4037     bicycle_parking
       4013     railway:traffic_mode
       4008     button_operated
       3987     nat_ref
       3842     comment
       3792     sub_bacia
       3699     outdoor_seating
       3692     parking:lane:both
       3659     contact:email
       3625     contact:facebook
       3606     fire_hydrant:type
       3575     snowmobile
       3570     room
       3569     self_service
       3552     sidewalk:surface
       3538     ski
       3531     old_operator
       3517     side
       3512     border_type
       3503     maxweight
       3503     source:surface
       3468     sidewalk:both
       3467     mapillary
       3420     sidewalk:width
       3407     operator:short
       3395     contact:instagram
       3374     embankment
       3342     utility
       3320     IPP:CodLote
       3274     traffic_signals:sound
       3268     motorcycle
       3209     is_in:country
       3182     source:alt_name
       3151     line_attachment
       3089     lamp_mount
       3081     destroyed:building
       3027     internet_access:fee
       2995     name:es
       2976     cemetery
       2970     parking:both
       2937     seamark:name
       2914     seats
       2905     tower:construction
       2783     artwork_type
       2763     wikimedia_commons
       2736     source:official_name
       2733     mobile
       2728     golf
       2656     parking:lane:right
       2611     parking:lane:left
       2603     delivery
       2584     implicit
       2567     overtaking
       2532     club
       2508     via
       2490     substance
       2475     addr:county
       2436     IJSN:administra
       2431     service_times
       2410     door
       2400     mitre:colour
       2397     related_law
       2392     handrail
       2383     IJSN:classeCnae
       2374     tidal
       2333     bridge:structure
       2322     ref:CNES
       2305     cycleway:both
       2291     reg_name
       2256     salt
       2232     building:flats
       2223     mofa
       2207     contact:mobile
       2192     icao
       2180     fuel:diesel
       2135     waste
       2131     payment:nfc
       2122     clothes
       2118     psv
       2114     stroller
       2105     sidewalk:both:surface
       2077     moped
       2060     shelter_type
       2043     pipeline
       2007     species:wikidata
       1952     is_in:suburb
       1928     proposed
       1920     information
       1898     payment:coins
       1889     currency:XBT
       1881     parking_space
       1879     trail_visibility
       1868     contact:whatsapp
       1867     garden:type
       1851     payment:lightning
       1849     meadow
       1839     maxspeed:forward
       1819     route_master
       1796     addr:neighbourhood
       1792     charge
       1783     maxspeed:backward
       1775     diameter_crown
       1759     trolley_wire
       1728     sac_scale
       1721     seamark:light:colour
       1712     seamark:light:character
       1705     payment:mastercard
       1700     social_facility
       1700     source:url
       1687     image
       1684     surveillance:type
       1683     plant:source
       1668     traffic_sign:forward
       1665     payment:visa
       1665     seamark:buoy_lateral:colour
       1661     seamark:buoy_lateral:category
       1661     seamark:buoy_lateral:shape
       1660     seamark:buoy_lateral:system
       1639     nat_name
       1631     destination:street
       1615     train
       1588     diameter
       1576     transit:lanes
       1557     roundtrip
       1554     note:pt
       1553     crossing:signals
       1545     protection_title
       1535     official_route
       1534     area:highway
       1529     swimming_pool
       1525     psv:lanes
       1521     dual_carriageway
       1514     athletics
       1513     ref:vatin
       1507     fuel:octane_87
       1498     gtfs:shape_id
       1491     payment:notes
       1491     protect_class
       1483     traffic_signals:vibration
       1461     disused:railway
       1442     design:incomplete
       1442     payment:telephone_cards
       1437     payment:onchain
       1429     fire_hydrant:position
       1419     name:etymology:wikidata
       1419     turn:lanes:forward
       1418     duration
       1418     surveillance
       1411     passenger_lines
       1407     support
       1403     coastline:survey_quality
       1398     time
       1393     ref:INEP
       1386     circumference
       1383     inscription
       1378     social_facility:for
       1366     camera:type
       1360     ferry
       1343     memorial
       1336     name:fr
       1318     abandoned:railway
       1306     military
       1304     length
       1301     traffic_sign:backward
       1298     surveillance:zone
       1296     busway:right
       1296     ramp
       1293     seamark:light:period
       1292     locked
       1291     flood_prone
       1284     seamark:rock:water_level
       1275     destination:forward
       1258     camera:mount
       1256     turn:lanes:backward
       1229     species:wikipedia
       1226     attraction
       1217     reservoir_type
       1212     second_hand
       1197     source:lit
       1196     carriageway_ref
       1196     gtfs_stop_desc
       1185     hazard
       1185     network:short
       1177     recycling_type
       1159     IPP:CODBAIRRO
       1157     fuel:ethanol
       1154     payment:lightning_contactless
       1151     ref:pmmcr:unico
       1137     toilets
       1132     seamark:topmark:shape
       1130     seamark:topmark:colour
       1128     grades
       1123     IPP:CODATIVIDA
       1123     IPP:CRE
       1123     IPP:DESIGNACAO
       1123     IPP:ENDNOVO
       1122     parking:both:orientation
       1122     uncontrolled
       1120     IPP:COD_INEP
       1120     police
       1119     IPP:BAIRRO
       1108     source:description
       1096     IPP:CHAVE
       1096     IPP:COD_SMA
       1095     triple_tower
       1092     bridge:name
       1085     int_name
       1083     access:lanes
       1083     fuel:etanol_comum
       1069     bridge:material
       1059     traffic_signals:foot
       1055     maxlength
       1048     crossing:barrier
       1044     mtb:scale
       1038     townhall:type
       1037     step_count
       1036     country
       1035     fitness_station
       1033     male
       1028     playground
       1027     advertising
       1024     plant:method
       1022     IPP:FLG_VALIDA
       1019     IPP:CodLogradouro
       1019     motorboat
       1013     building:min_level
       1010     parking:right
       1008     brand:short
        995     IPP:CodFavela
        994     source:old_name
        992     addr:door
        992     enforcement
        990     mtb:scale:imba
        982     plant:output:electricity
        982     subway
        978     source:law
        971     bus:lanes
        946     cutting
        941     source:damage
        923     parking:left
        919     addr:hamlet
        905     is_in
        903     check_date:currency:XBT
        891     colour:text
        890     traffic_signals:countdown
        889     colour:back
        887     irrigation
        880     trolleybus
        879     colour:arrow
        878     isced:level
        873     beauty
        870     lanes:bus
        856     ref:pmf
        854     passenger_information_display
        851     female
        844     maxspeed:practical
        843     maxwidth
        829     currency:BRL
        823     4wd_only
        817     maxspeed:bus
        815     fuel:gasoline_87
        814     fax
        805     source:position
        802     taxon:family
        795     payment:contactless
        789     IPP:CodQuadra
        784     payment:pix
        770     toilets:disposal
        765     source:tracer
        764     drinking_water
        760     source:addr:housenumber
        759     goods
        759     incorrect:name
        756     railway:preferred_direction
        752     source:height
        745     line
        739     IPP:ENDERECO
        737     rooms
        736     capacity:disabled
        735     compressed_air
        730     motorroad
        726     turning_circle
        719     source_ref
        719     studio
        718     species:pt
        714     inep
        711     woodland
        704     source:zoomlevel
        696     diet:vegetarian
        696     lanes:psv
        696     seamark:light:range
        691     levels
        691     operator:ref:vatin
        689     busway:left
        688     community_centre
        684     preschool
        681     transformer
        675     IPP:CODLINFERR
        673     elevation
        669     brewery
        665     camera:direction
        660     trade
        655     disused
        655     source:sidewalk
        653     destination:backward
        653     parking:right:orientation
        653     roof:direction
        650     artist_name
        650     countdown_timer:vehicles
        648     junction:ref
        642     lamp:power
        639     park_ride
        634     street:name
        633     line_management
        631     diet:vegan
        629     is_in:neighbourhood
        623     source:oneway
        619     IPP:TELEFONES
        618     maxspeed:advisory
        616     abandoned
        612     min_age
        612     ship
        609     name:signed
        609     resource
        608     parking:condition:right
        605     source:access
        603     name:ru
        594     water_source
        588     station
        585     maxweight:signed
        582     armrest
        578     mitre:material
        572     basin
        568     bridge:support
        558     internet_access:ssid
        558     railway:atc
        556     unisex
        554     operator:website
        553     payment:cards
        539     fuel:gasoline
        536     building:levels:underground
        535     url
        533     addr:flats
        531     state
        530     crossing:light
        525     sidewalk:right:surface
        524     operator:phone
        524     orientation
        524     payment:maestro
        523     aerialway
        521     grassland
        520     incorrect:junction
        520     stars
        518     railway:ref
        517     maxspeed:bicycle
        515     flow_rate
        511     source:maxspeed:bicycle
        508     toilets:wheelchair
        507     tracks
        505     lamp:type
        505     light_rail
        504     disused:amenity
        504     seamark:light:height
        500     service:vehicle:car_repair
        497     floating
        497     is_in:city
        496     district:type
        494     seamark:light:reference
        493     disused:shop
        487     crossing:bell
        487     traffic_sign:direction
        486     check_date:opening_hours
        481     lcn
        479     ref:CNUC
        479     repair
        477     conveying
        477     fuel:octane_95
        476     maxspeed:lanes
        473     demolished:building
        472     parking:left:orientation
        470     location:transition
        469     source:noname
        465     GEOCODIG_M
        465     magic_wand
        457     payment:visa_electron
        457     seamark:notice:system
        456     changing_table
        453     height_trunk
        450     vending
        449     ref:Funai
        446     estimated_age
        446     opening_hours:covid19
        446     seamark:distance_mark:category
        446     seamark:distance_mark:distance
        446     seamark:distance_mark:units
        446     unsigned
        440     species:en
        439     payment:american_express
        438     oneway_type
        435     canoe
        435     cycleway:left:oneway
        434     ruins
        433     bicycle_rental
        433     payment:coinos
        432     seamark:notice:1:system
        432     seamark:notice:2:system
        431     wholesale
        430     seamark:notice:bank
        429     seamark:notice:function
        428     service:vehicle:oil_change
        426     parking:condition:both
        426     wood
        423     service:bicycle:repair
        421     seamark:notice:1:function
        421     seamark:notice:2:function
        420     cycleway:est_width
        420     payment:electronic_purses
        420     seamark:notice:1:category
        420     seamark:notice:1:impact
        420     seamark:notice:2:category
        420     seamark:notice:2:impact
        418     lifeguard
        418     seamark:notice:1:bank
        418     seamark:notice:2:bank
        416     FIXME
        415     organic
        413     population:man
        413     population:woman
        410     electrified:rail
        410     payment:visa_debit
        409     destination:ref:to
        408     IPP:CODIGO
        403     seamark:landmark:category
        402     name:de
        401     source:ele
        398     destination:colour
        397     lighthouse:shape
        391     reg_ref
        389     harbour
        389     parking:condition:left
        388     is_in:state_code
        386     depot
        385     seamark:mooring:category
        384     addr:full
        383     blind
        383     livestock
        380     manhole
        379     rental
        378     mooring
        378     seamark:light:group
        373     surface:lanes
        372     cycleway:right:oneway
        372     kerb:height
        368     playground:theme
        367     seamark:tank:category
        366     tourist_bus
        364     bar
        363     railway:etcs
        362     destination:symbol
        362     fire_hydrant:diameter
        361     DER:RS:zona_de_fiscalizacao
        360     name:oym
        359     IPP:CADASTRAL
        356     IPP:COD_QUADRA
        356     heritage
        352     id
        349     mtb:scale:uphill
        348     addr:milestone
        348     name:ja
        348     opening_date
        345     visibility
        342     hgv:conditional
        341     light:lit
        340     disused:highway
        340     iata
        340     light:method
        339     language:en
        337     DER:RS:modalidade
        334     PrefRecife:escola_codigo
        334     PrefRecife:mec_codigo
        334     fuel:cng
        333     PrefRecife:escola_tipo
        333     is_in:country_code
        329     name:etymology:wikipedia
        329     reef
        328     motor_vehicle:conditional
        327     authentication:app
        327     level_name
        325     name:he
        325     status
        321     check_date:surface
        320     footway:surface
        320     noref
        320     not:name
        319     museum
        318     recycling:paper
        317     max_age
        316     lift_gate:type
        315     alt_name_1
        311     seamark:wreck:category
        307     seamark:harbour:category
        305     managed
        302     indoor_seating
        301     agricultural
        300     communication:microwave
        300     parking:both:restriction
        299     highspeed
        298     two_sided
        297     ruins:building
        296     emergency:phone
        296     fishing
        295     cycleway:surface
        294     diplomatic
        291     bicycle:conditional
        290     flag:type
        288     couplings
        287     cargo
        287     distrito
        286     contact:twitter
        286     fuel:diesel_S10
        285     decreto:ponto
        282     symbol
        281     check_date:shelter
        279     microbrewery
        277     addr:subdistrict
        275     abandoned:highway
        273     marker
        272     description:pt
        270     source:motorboat
        269     addr:floor
        267     surface:grade
        264     payment:diners_club
        264     service:bicycle:retail
        263     IBGE:tipo
        263     theatre:type
        262     automated
        262     communication:radio
        262     name:zh
        260     payment:cryptocurrencies
        259     depth
        259     longitude
        258     latitude
        258     railway:end_date
        258     recycling:plastic
        257     oneway:conditional
        255     end_date
        255     wikipedia:en
        254     ref:Inmetro
        253     hairdresser
        250     phone_1
        249     communication:television
        248     interval
        247     damage:event:wikidata
        244     board_type
        244     city_block:network
        244     damage
        244     not:brand:wikidata
        244     ramp:wheelchair
        244     source:restriction
        243     DER:RS:categoria_rodoviaria
        243     airmark
        243     draft
        243     network:type
        243     wdb:source
        242     is_sidepath:of:name
        241     seamark:landmark:function
        240     is_sidepath:of
        239     abandoned:electrified
        239     display
        239     est_width
        235     addr:description
        234     beacon:type
        233     language:es
        232     countdown_timer:pedestrians
        231     flag
        231     source:housenumber
        231     target
        230     beacon:frequency
        230     farmland
        228     beacon:code
        228     contact:fax
        228     is_in:province
        228     pt:bicycle_parking
        227     panoramax
        226     construction:railway
        226     maxweight:conditional
        226     switch
        224     IPP:Codigo
        224     hoops
        222     model
        222     old_postal_code
        219     aerodrome:type
        219     seamark:buoy_special_purpose:colour
        219     taxi_vehicle
        218     source:draft
        217     maxlength:conditional
        217     seamark:buoy_special_purpose:shape
        216     animal
        215     name:lt
        213     lanes:bus:conditional
        211     GEOCODIG_D
        210     fountain
        208     IPP:OBS
        206     architect
        206     dual_gauge
        206     name:ko
        204     alt_name:en
        204     damage:event
        204     not:operator:wikidata
        203     disused:building
        203     historic:railway
        203     source:opening_hours
        201     contact:youtube
        201     manufacturer
        201     parking:both:fee
        200     cash_in
        200     railway:cbtc
        199     navigationaid
        197     heritage:operator
        197     service:vehicle:tyres
        196     guest_house
        195     grass
        195     gtfs:stop_id
        195     pav
        194     abandoned:usage
        194     driving_side
        193     destination:lanes
        193     monitoring:weather
        193     recycling:scrap_metal
        192     abandoned:start_date
        192     operator:en
        191     training
        190     damage:type
        190     pillar:type
        190     service:vehicle:electrical
        189     diet:meat
        189     name:uk
        186     except
        186     name:ar
        186     service:vehicle:diagnostics
        186     site
        183     bdouble
        182     crossing:saltire
        182     payment:cheque
        181     abandoned:end_date
        181     abandoned:gauge
        181     recycling:glass_bottles
        180     parking:condition:right:time_interval
        180     toilets:position
        179     left:country
        179     maxspeed:motorcar
        179     name:bg
        179     name:ka
        179     payment:bitcoin
        179     railway:ballastless
        179     right:country
        179     service:vehicle:used_car_sales
        178     busway
        178     recycling:cans
        177     rating
        176     abandoned:name
        175     cap:colour
        175     garden:style
        175     maxspeed:motorcycle
        174     parking:lane:both:parallel
        173     fid
        173     service:vehicle:brakes
        172     recycling:glass
        171     baby_feeding
        170     bollard
        170     name:sr
        170     polling_station
        169     cooling:method
        169     service:bicycle:pump
        169     tram
        167     subject:wikidata
        165     voltage:primary
        164     maxstay
        163     shoes
        163     sidewalk:left:surface
        161     hiking
        161     name:kk
        161     not:highway
        161     school
        161     taxi
        160     animal_breeding
        160     handrail:right
        160     light:count
        160     seamark:radar_reflector
        160     wifi
        158     parking:right:restriction
        158     phases
        158     voltage:secondary
        157     ref:cue
        156     bonnet:colour
        156     name:lv
        156     payment:discover_card
        154     hgv:lanes
        154     proposed:highway
        153     alt_name1
        153     foot:conditional
        153     parking:lane:right:parallel
        153     vaccination
        152     neighborhood
        151     fuel:gasolina_comum
        151     name:it
        150     handrail:left
        150     maxspeed:conditional
        150     service:vehicle:inspection
        149     description:name
        149     frontage_road
        149     name:mk
        149     payment:voucher_meal
        148     OBJECTID
        148     Shape_Leng
        148     payment:account_cards
        148     recycling:plastic_bottles
        147     hov
        147     restriction:conditional
        145     pump
        144     fuel:gasolina_aditivada
        144     network:class
        142     date
        142     radar_reflector
        141     crossing:chicane
        141     fuel:octane_91
        141     monitoring:ombrometer
        141     service:vehicle:batteries
        140     hazard_type
        140     paving_stones:shape
        140     seamark:notice:category
        139     dance:teaching
        139     name_1
        138     passing_places
        137     IPP:Fonte
        137     service:bicycle:second_hand
        136     consulate
        136     designation
        136     embassy
        136     service:bicycle:cleaning
        135     exit
        135     fuel:biodiesel
        135     railway:preserved
        135     source:narrow
        134     name:ca
        132     light:shape
        132     service:vehicle:repairs
        132     toilets:access
        131     hazard_prone
        131     message
        131     name:eo
        131     width:lanes
        130     destination:colour:text
        130     payment:girocard
        130     source:building:levels
        130     source:width
        130     square
        130     telecom
        129     placement
        128     destination:colour:arrow
        128     maxheight:physical
        128     tessellate
        128     weather:rain_gauge
        127     object:street
        125     district
        125     fuel:e85
        125     private
        124     IPP:CodTrecho
        124     object:city
        124     payment:dkv
        124     payment:uta
        124     source:image
        123     order
        123     parking:left:restriction
        123     related_law:url
        123     service:vehicle:new_car_sales
        123     source:source
        122     IPP:CodNP
        122     addr:province
        122     minspeed
        122     park
        122     zone
        121     FID
        121     sett:style
        120     cycleway:left:conditional
        120     name:fa
        120     related_law:name
        120     sett:shape
        119     postal_code:side:even
        119     postal_code:side:odd
        119     ref:DETRO
        119     ref:operator
        117     fuel
        117     styleUrl
        116     family
        116     seamark:platform:category
        115     bus:service
        115     parking_lane
        114     name:be
        114     opening_hours:signed
        114     shape_leng
        113     addr:block
        112     amenity_1
        112     objectid_1
        112     recycling:plastic_packaging
        112     shape_le_1
        111     light:colour
        111     name:hy
        111     postal_code:left
        111     stairs
        110     access:conditional
        110     addr:complement
        110     nome
        110     seamark:beacon_lateral:category
        110     seamark:pipeline_submarine:product
        110     source:river
        109     name:pl
        109     seamark:beacon_lateral:colour
        109     seamark:beacon_lateral:system
        108     alt_name:pt
        107     bus_bay
        107     diet:gluten_free
        107     thermometer
        106     IJSN:tipoUnidad
        106     lockable
        106     protected
        105     chave
        105     facebook
        105     kerb:left
        105     parking:orientation
        105     source:hgv
        105     wikipedia:pt
        104     bottle
        104     branch:type
        104     fast_food
        104     kerb:right
        103     aerodrome
        103     coach
        103     handrail:center
        103     name:mr
        103     seamark:information
        102     IBGE:CD_GEOCODB
        102     highchair
        102     name:cs
        102     name:kn
        102     name:th
        102     payment:pikepass
        102     surface:colour
        102     tents
        102     wikimapa:author
        101     bus:lanes:backward
        101     ele:source
        101     emergency:exit
        101     faces
        101     flag:name
        100     ele:msl
        100     historic:civilization
        100     postal_code:right
        100     website:menu
         99     internet_access:operator
         99     recycling:cardboard
         99     restaurant
         99     service:bicycle:rental
         98     contact:linkedin
         98     peak
         98     seamark:pylon:category
         97     fuel:diesel_s10_comum
         96     abandoned:building
         96     communication:gsm
         96     taxi:colour
         95     shoulder:right
         95     sport_1
         94     building_1
         94     note:alt_name
         94     service:bicycle:diy
         94     source:building
         94     source:lanes
         94     tag
         93     building:architecture
         93     name:tg
         93     note:name
         92     food
         92     ref:EPTC
         92     service:bicycle:parts
         92     sorting_name
         91     addr:number
         91     name:nl
         91     payment:app
         90     axle_load
         90     panoramax:0
         90     railway:switch
         90     seamark:notice:impact
         90     source:length
         90     steps
         89     fuel:e10
         89     taxi:prefix
         88     classeCnae
         88     flag:wikidata
         87     archaeological_site
         87     name:la
         86     access:bus
         86     bus:lanes:forward
         86     currency:others
         86     maxheight:signed
         86     sett:pattern
         86     subject
         86     toilets:handwashing
         85     communication:satellite
         85     service:vehicle:painting
         85     source_1
         84     brand:en
         84     fuel:octane_98
         83     addr:block_number
         83     alt
         83     disused:name
         83     payment:apple_pay
         83     shoulder:surface
         82     Bairro
         82     description:payment
         82     monorail
         82     noaddress
         82     recycling:green_waste
         82     service:vehicle:car_parts
         81     brand:es
         81     health
         81     lgbtq
         81     light:flash
         81     name:uz
         81     seamark:national_information
         80     seamark:pylon:water_level
         79     fuel:alcohol
         79     old_brand
         79     ref:SMTR_AUX
         79     seamark:restricted_area:restriction
         79     source:goods
         78     cycleway:left:est_width
         77     uic_name
         77     width:carriageway
         77     zone:maxspeed
         76     alt_name:es
         76     fuel:HGV_diesel
         76     map_type
         76     obstacle
         76     phone:mobile
         75     climbing
         75     devices
         75     name:os
         75     name:sv
         75     proposed:landuse
         75     ref:PBH
         75     seamark:fishing_facility:category
         75     service:vehicle:air_conditioning
         75     tunnel:name
         74     caravans
         74     diocese
         74     divipola
         74     parking:condition:right:maxstay
         73     company
         73     holding_position:type
         73     parts
         72     cycleway:right:est_width
         72     name:right
         72     overgrown
         72     paving_stones:pattern
         72     power_supply
         72     ref:jgs:irreg
         71     agrarian
         71     name:left
         71     parking:condition:right:default
         71     seamark:radio_station:category
         71     seamark:signal_station_warning:category
         71     weather_protection
         70     light:direction
         70     lock
         70     railway:turnout_side
         70     screen
         70     seamark:light:1:colour
         70     shower
         70     sidewalk:right:tactile_paving
         70     surface:note
         69     motorcycle:conditional
         69     name:el
         69     parking:right:fee
         69     seamark:light:1:character
         69     source:loc_name
         68     animal_keeping:type
         68     collection_times
         68     cycleway:left:source
         68     payment:google_pay
         68     seamark:small_craft_facility:category
         67     buildingpart
         67     name:sh
         67     railway:start_date
         67     service:vehicle:alignment
         66     capacity:parent
         66     education
         66     fuel:diesel_s500_comum
         66     parking:condition:left:time_interval
         66     railway:historic
         66     recycling:aluminium
         66     seamark:buoy_cardinal:category
         66     seamark:buoy_cardinal:colour
         66     seamark:buoy_cardinal:colour_pattern
         65     beds
         65     bicycle_road
         65     castle_type
         65     map_size
         65     nudism
         65     other_name
         65     parking:lane
         65     parking:left:fee
         65     seamark:buoy_cardinal:shape
         65     seamark:buoy_special_purpose:colour_pattern
         65     source:fixme
         64     artist:wikidata
         64     maritime
         64     name:br
         64     name:tr
         64     service:vehicle:truck_repair
         64     source:boundary
         64     subject:wikipedia
         64     waterway:information
         64     waterway:information:pt
         63     diet:local
         63     governance_type
         63     law
         63     name:gn
         63     seamark:light:1:range
         63     seamark:light:1:sector_end
         63     seamark:light:1:sector_start
         63     source:cycleway
         62     3dr:type
         62     alt_ref
         62     busway:backward:right
         62     capacity:women
         62     coffee
         62     couplings:type
         62     diet:lactose_free
         62     seamark:light:1:height
         62     seamark:light:1:period
         61     IPP:FONTE
         61     IPP:RAMAL
         61     boules
         61     crossing:supervision
         61     free_flying:site
         61     fuel:octane_80
         61     seamark:buoy_isolated_danger:colour
         61     seamark:buoy_isolated_danger:colour_pattern
         61     seamark:buoy_isolated_danger:shape
         61     strapline
         60     IPP:CODESTFERR
         60     backcountry
         60     dance:style
         60     destination:en
         59     FUNAI:GRUPOS
         59     fuel:lpg
         59     genus:wikidata
         59     group_only
         59     parking:both:staggered
         59     protected_area
         59     seamark:building:function
         59     seamark:buoy_special_purpose:category
         59     service:vehicle:body_repair
         58     FUNAI:AREA_HA
         58     FUNAI:LINKAREA
         58     FUNAI:SITUACAO
         58     IBGE:BACIA
         58     ISO3166-2
         58     currency:USD
         58     description:en
         58     destination:street:to
         58     emergency_shelter
         58     emergency_shelter:ref
         58     fixme:name
         58     note:en
         58     parking:lane:left:parallel
         57     FUNAI:CODAREA
         57     FUNAI:NOME_AREA
         57     FUNAI:TITULO
         57     destination:to
         57     fuel:octane_87_aditivada
         57     hour_on
         57     name:ur
         57     railway:position
         57     roof:angle
         56     earth_bank
         56     faculty
         56     fuel:biogas
         56     fuel:octane_92
         56     gate:type
         56     minibus
         56     monitoring:air_quality
         56     plant
         56     roof:type
         56     seamark:restricted_area:category
         56     service:bicycle:Bicycle_Sales_and_Service
         55     animated
         55     fuel:diesel_s10
         55     hour_off
         55     inlet
         55     name:eu
         55     observatory:type
         55     par
         55     seamark:light:category
         55     sidewalk:left:tactile_paving
         55     source:smoothness
         55     women
         54     IPP:ID
         54     building:floor
         54     incorrect:highway
         54     name:hi
         54     sport_2
         53     PKUID
         53     is_in:street
         53     path
         53     pumping_station
         53     railway:name
         53     ref:BRTRio
         53     ref:UY:ruee
         53     ref:UY:rule_est
         53     seamark:pipeline_submarine:category
         53     storage_tank:type
         52     CEP
         52     bicycle_parking:pt
         52     cycleway:width
         52     date_off
         52     date_on
         52     denomination:wikidata
         52     light:tilt
         52     lunch
         52     parking:lane:left:capacity
         52     recycling:wood
         52     remotely_controllable
         52     shoulder:smoothness
         51     IJSN:tipoAgrup
         51     breakfast
         51     bus_route
         51     name:ce
         51     name:eme
         51     name:gl
         51     park:type
         51     pets
         51     pole
         51     repeat_on
         51     sanitary_dump_station
         51     state_capital
         50     CLC:id
         50     CLC:year
         50     FUNAI:AREAETNO
         50     FUNAI:CDOC
         50     FUNAI:PERIMETRO
         50     ele:SIRGAS2000
         50     health_facility:type
         50     heritage:ref
         50     luminous
         50     manufacturer:type
         50     payment:elo
         50     recycling:batteries
         50     transit:lanes:forward
         49     FUNAI:DATADOC
         49     FUNAI:DOCUMENTO
         49     FUNAI:ETAPA
         49     FUNAI:EXTENSO
         49     FUNAI:FLUXO
         49     FUNAI:GID
         49     abandoned:operator
         49     communication:mobile
         49     denomination:wikipedia
         49     geological
         49     internet_access:password
         49     military_service
         49     name:lanes
         49     old_addr:housenumber
         49     seamark:beacon_special_purpose:colour
         49     source:motor_vehicle
         48     IPP:ORGAO1
         48     IPP:TUTELA1
         48     drink:beer
         48     groundcheck
         48     rock
         48     service:bicycle:chain_tool
         48     source:cycleway:right
         47     FUNAI:ADR
         47     IPP:UC
         47     artwork_subject
         47     motorcycle:repair
         47     parking:lane:right:capacity
         47     pilotage
         47     ref_name
         47     wheelchair:description
         46     MCC
         46     MNC
         46     abbr_name
         46     bridge:ref
         46     building:units
         46     contact:webcam
         46     ele:wgs84
         46     inscription_date
         46     is_in:town
         46     logistics
         46     seamark:light:status
         46     service:bicycle:sales
         46     wikiversity
         45     IPP:Classe
         45     campus
         45     classification
         45     cycleway:separation
         45     laboratory
         45     name:mn
         45     official_name:en
         45     recycling:waste
         45     seamark:bridge:clearance_height
         45     taxon:subfamily
         44     bath:open_air
         44     ibgE:CD_ADMINIS
         44     min_level
         44     name:ku
         44     polyline_img_type
         44     railway:old_name
         44     rapids
         44     seamark:status
         44     service:bicycle:tools
         44     timezone
         44     tree
         43     c├│d
         43     footway:left
         43     fuel:electricity
         43     hazmat
         43     recycling:metal
         43     ref:der-es
         43     seamark:radio_station:callsign
         43     sidewalk:right:bicycle
         43     timetable
         42     2016olympicgames
         42     diet:non-vegetarian
         42     lanes:reversible
         42     living_street
         42     name:ro
         42     stranded
         41     community_centre:for
         41     couplings:diameters
         41     cycleway:left:separation
         41     fuel:diesel_S500
         41     name:ta
         41     platforms
         41     recreation
         41     roof
         41     service:vehicle:suspension
         41     tomb
         41     was:amenity
         40     CD_GEOCODB
         40     IPP:ATOLEGAL
         40     added_by
         40     communication:antenna
         40     demolished:building:levels
         40     fuel:1_25
         40     lanes:reversible:type
         40     lit:perceived
         40     operational_status
         40     parking:fee
         40     psv:lanes:forward
         40     restriction:hgv
         40     seamark:conspicuity
         40     service:vehicle:motor
         40     source:destination
         39     IPP:AREA_PLANE
         39     IPP:CODRA
         39     REGIME
         39     alcohol
         39     area:aeroway
         39     ascent
         39     commercial
         39     cutline
         39     destination:lang:en
         39     ethnic_group
         39     fuel:octane_100
         39     heritage:name
         39     official_name:pt
         39     psv:lanes:backward
         39     public
         39     seamark:radar_transponder:category
         39     service:vehicle:wheels
         39     source:material
         39     turn
         38     IBGE:NASCENTE
         38     crane:type
         38     depth:dredged
         38     depth:dredged:year
         38     farmyard
         38     fuel:GTL_diesel
         38     instagram
         38     name:bn
         38     name:id
         38     name:oc
         38     oneway:bus
         38     parking:condition:left:maxstay
         38     payment:sem_parar
         38     ramp:bicycle
         38     seamark:radar_transponder:group
         38     source:cycleway:left
         38     source:vehicle
         38     waterway_1
         37     addr:postcode:odd
         37     building:height
         37     currency:EUR
         37     monitoring:water_level
         37     name:az
         37     name:et
         37     name:hu
         37     name:sk
         37     railway:usage
         37     recycling:cooking_oil
         37     ref:SE_RS
         37     seamark:bridge:category
         37     source:geometry:date
         37     tunnel:ref
         36     IPP:Telefone
         36     abandoned:amenity
         36     cycleway:name
         36     damage:date
         36     diet:halal
         36     fuel:gasolina
         36     indoor:level
         36     listed_status
         36     name:no
         36     name:vi
         36     parking:condition:right:vehicles
         36     perimeter
         36     railway:local_operated
         36     railway:switch:electric
         36     ref:log
         36     seamark:beacon_lateral:shape
         36     sidewalk:right:foot
         36     sidewalks
         36     source:addr:street
         36     storage
         35     NOME
         35     animal_keeping
         35     animal_shelter
         35     esperanto
         35     exit_to
         35     full_name
         35     golf_cart
         35     line_arrangement
         35     long_name
         35     mobile_phone:repair
         35     motorcycle:parts
         34     disused:public_transport
         34     monument
         34     name:hr
         34     service:press
         34     source:railway
         34     transit:lanes:backward
         33     bath:type
         33     climbing:bolted
         33     communication:internet
         33     communication:lte
         33     diplomatic:services:citizen_services
         33     elevator
         33     leisure1
         33     maxheight:marine
         33     name:da
         33     name:fi
         33     old
         33     railway:position:exact
         33     recycling:garden_waste
         33     seamark:calling-in_point:traffic_flow
         33     source:maxlength
         33     surface_survey
         33     ufrn:codigo_unidade
         33     voltage:tertiary
         32     IPP:Endereco
         32     IPP:REGIAO_ADM
         32     church:type
         32     generator:plant
         32     headframe
         32     lanes:reversible:backward
         32     lanes:reversible:forward
         32     name:co
         32     name:tt
         32     name:zh-Hans
         32     name:zh-Hant
         32     outdoor
         32     parking:lanes:both
         32     paved
         32     recycling:electrical_appliances
         32     ref:insular
         32     seamark:beacon_isolated_danger:colour
         32     seamark:beacon_isolated_danger:colour_pattern
         32     source:depth
         32     survey_point:structure
         31     baby
         31     capital
         31     car_wash
         31     descent
         31     diet:kosher
         31     disused:aeroway
         31     dock
         31     electronics_repair
         31     lanes:bus:backward
         31     name:qu
         31     pets_allowed
         31     seamark:anchorage:category
         31     service:bicycle:bicycle
         31     service:bicycle:service
         31     service:vehicle:glass
         31     state_code
         31     street_cabinet
         31     water_supply
         30     alias
         30     communication
         30     department
         30     diet:pescetarian
         30     dwc:occurrenceID
         30     fixme:surface
         30     id:clearchannel
         30     iphan:inscription_date
         30     landuse1
         30     monitoring:temperature
         30     motorcycle:sales
         30     mtb:type
         30     name:sq
         30     old_website
         30     oneway:forward:conditional
         30     opening_hours:url
         30     phone:2
         30     production
         30     rack
         30     railway:gauge
         30     recording:manually
         30     sale
         30     seamark:cable_overhead:category
         30     seamark:calling-in_point:orientation
         30     seamark:shoreline_construction:category
         30     source:postcode
         30     waterway:type
         29     CODIGO
         29     FNODE_
         29     IJSN:setor
         29     addr:room
         29     brunch
         29     cycleway:right:segregated
         29     dinner
         29     language:fr
         29     local_name
         29     multipolygon
         29     name:ms
         29     nature
         29     not:maxspeed
         29     payment:mastercard_contactless
         29     piste:type
         29     razed:railway
         29     recycling:newspaper
         29     resort
         29     since
         29     unit
         29     zoo
         28     GNS:id
         28     Id
         28     bath:sand_bath
         28     blood:whole
         28     climbing:sport
         28     drink:coffee
         28     footway:right
         28     kml_folder
         28     name:yo
         28     not:network:wikidata
         28     official_name:es
         28     priority_road
         28     ref:SMTR
         28     ref:iphan
         28     source:maxspeed:forward
         28     source:operator
         28     vehicle:conditional
         27     abandoned:old_name
         27     destination:ref:to:backward
         27     fire_hydrant:pressure
         27     handicap
         27     lanes:psv:times
         27     mattress
         27     na
         27     name:bs
         27     name:ia
         27     name:io
         27     name:yi
         27     not:ref
         27     payment:paypal
         27     portable
         27     ramp:luggage
         27     sidewalk:right:kerb
         27     siren:purpose
         26     IPP:UCSTATUS
         26     access:taxi
         26     demolished:roof:shape
         26     diet:healthy
         26     have_riverbank
         26     interval:source
         26     kiosk
         26     lanes:psv:forward:conditional
         26     language:de
         26     laundry_service
         26     left:8
         26     motorcar:conditional
         26     name:ml
         26     parish
         26     recording:automated
         26     shoulder:bicycle
         26     source:capacity
         26     speed_camera
         26     survey_date
         25     EntityHand
         25     IPP:CATEGORIA
         25     books
         25     comdephaapasa:inscription_date
         25     cycle_network
         25     demolished:highway
         25     detour
         25     female_owner
         25     fixme:type
         25     forestry
         25     fuel:etanol_aditivado
         25     hot_water
         25     name:abbreviation
         25     name:am
         25     name:ba
         25     name:pa
         25     operator:es
         25     operator:pt
         25     parking:right:markings
         25     payment:credit_card
         25     postal_code:country
         25     ramp:stroller
         25     shop_1
         25     sidewalk:both:bicycle
         25     situation
         25     source:addr:name
         25     traffic_signals:arrow
         24     2016olympicgames:zone
         24     building:max_level
         24     closed
         24     communication:4g
         24     diet:chicken
         24     fuel:GNV
         24     fuel:adblue
         24     iphan:status
         24     is_in:county
         24     men
         24     name:af
         24     name:ay
         24     name:cv
         24     name:is
         24     name:sl
         24     name:tuo
         24     parking:wheelchair
         24     recycling:clothes
         24     reference
         24     road_marking
         24     seamark:conveyor:category
         24     seating
         24     service:vehicle:transmission_repair
         24     sport_3
         24     telecom:medium
         24     trolley_wire:lanes
         24     truck
         23     Area
         23     benches
         23     building:prefabricated
         23     culvert
         23     cycleway:both:lane
         23     cycleway:foot
         23     diplomatic:services:passeport
         23     disused:leisure
         23     drink:distilled
         23     estruturas
         23     hvg
         23     internet
         23     is_in:district
         23     max_level
         23     maxheight:legal
         23     network_1
         23     pool
         23     recycling:magazines
         23     seamark:obstruction:category
         23     trolley_wire:backward
         22     Distrito Postal
         22     SubClasses
         22     aerialway:occupancy
         22     building:recuos
         22     capacity:persons
         22     check_date:crossing
         22     check_date:smoothness
         22     children
         22     communication:gsm-r
         22     dangerous_place
         22     destroyed:highway
         22     drive_in
         22     duty_free
         22     fuel:Gasolina
         22     hand_cart
         22     highway:category:pl
         22     highway_authority_ref
         22     incorrect:ref
         22     iphan:classification
         22     kids_area
         22     kitchen
         22     lanes:psv:forward
         22     license_classes
         22     mechanical
         22     monitoring:traffic
         22     name:ug
         22     network:website
         22     parking:left:markings
         22     payment:picpay
         22     railway:switch:resetting
         22     shortest_name
         22     sides
         22     sign
         22     skateboard
         22     static_caravans
         21     Green
         21     anac
         21     anac:link
         21     big_rigs
         21     building:roof:colour
         21     church
         21     comercio
         21     currency:Real
         21     demolished:natural
         21     depth:dangerous
         21     diet:fish
         21     drink:cocktail
         21     drink:wine
         21     fuel:diesel_s10_aditivado
         21     information:pt
         21     lanes:backward:conditional
         21     left:4
         21     name:cy
         21     name:ga
         21     name:ne
         21     name:source
         21     parking:right:access
         21     payment:voucher
         21     railway:track_ref
         21     recycling:electrical_items
         21     recycling:paper_packaging
         21     ref:sisa_codigo
         21     restriction:hgv:conditional
         21     seamark:mooring:shape
         21     todo
         21     walking
         21     water:location
         20     IGAM:BACIA
         20     access:temporary
         20     alt_name_2
         20     animal_boarding
         20     bairro
         20     blood:platelets
         20     building:roof
         20     changing_table:fee
         20     communication:3g
         20     condition
         20     destroyed:bridge
         20     dish_washing
         20     disused:power
         20     fireplace
         20     locomotive:type
         20     maxspeed:goods
         20     name:ast
         20     name:fy
         20     name:lb
         20     name:sw
         20     official_ref
         20     old_addr:street
         20     parking:condition:both:maxstay
         20     payment:hipercard
         20     picnic_benches
         20     river
         20     seamark:light:1:group
         20     service:vehicle:transmission
         20     value
         20     verge
         20     water:quality
         20     wood:density
         19     CODE
         19     Layer
         19     Profundidade
         19     age
         19     camp_type
         19     check_date:bench
         19     city
         19     climbing:rock
         19     cold_water
         19     consulting
         19     destination:ref:to:forward
         19     diet:organic
         19     disused:gauge
         19     disused:service
         19     dryer
         19     general_room
         19     iphan:tombo
         19     lanes:bus:forward
         19     laundry
         19     maxspeed:psv
         19     name:bpy
         19     name:dv
         19     name:mi
         19     name:mt
         19     name:pnb
         19     name:se
         19     name:war
         19     note_pt
         19     oneway:backward:conditional
         19     owner:wikidata
         19     payment:jcb
         19     rail
         19     railway:milestone:catenary_mast
         19     ref:ZonaSanitaria
         19     ref:cuie_codigo
         19     runway
         19     sanitary_dumpstation
         19     seamark:cable_submarine:category
         19     soccer
         19     source:maxspeed:backward
         19     species:br
         19     theatre:genre
         19     tool
         19     traffic_signals:forward
         19     video
         19     window
         18     IDTUP
         18     address
         18     bus:conditional
         18     changing_table:location
         18     check_date:capacity
         18     contact:skype
         18     crane:mobile
         18     cycleway:left:width
         18     dist:blue
         18     dist:red
         18     dist:white
         18     emergency_ward_entrance
         18     flashing_lights
         18     genus:en
         18     hgv:lanes:forward
         18     horse_scale
         18     maxaxles
         18     mimics
         18     name:ceb
         18     name:li
         18     name:nn
         18     parking:condition:both:time_interval
         18     parking:condition:left:vehicles
         18     payment:boleto_bancario
         18     pe:ubigeo
         18     pet
         18     psv:lanes:conditional
         18     rcn_ref
         18     ref:cuise
         18     seamark:light:2:colour
         18     service:vehicle:sound
         18     sidewalk:left:width
         18     sidewalk:right:width
         18     source:photo
         18     toilets:menstrual_products
         18     tree_form
         18     washing_machine
         18     way
         17     addr:suburb_1
         17     airside
         17     automatic_door
         17     canal
         17     cinema:3D
         17     classe
         17     community
         17     contact:telegram
         17     cycleway:crossing:markings
         17     cycleway:right:lane
         17     destroyed
         17     diet:lacto_vegetarian
         17     display:analog
         17     donation:compensation
         17     feeding:for
         17     feeding:type
         17     golf:course
         17     inside
         17     is_in:housenumber
         17     is_in:subdistrict
         17     mapillary:map_feature
         17     name:fo
         17     name:gd
         17     name:kw
         17     name:my
         17     name:or
         17     name:rm
         17     name:sco
         17     name:su
         17     object:suburb
         17     panoramax:1
         17     payment:boleto
         17     payment:ep_geldkarte
         17     payment:none
         17     payment:shell_box
         17     payment:veloe
         17     recycling:rubble
         17     rotor:diameter
         17     sccn
         17     school:gender
         17     seamark:beacon_special_purpose:category
         17     seamark:berth:name
         17     seamark:buoy_lateral:colour_pattern
         17     seamark:light:2:character
         17     seamark:navigation_line:category
         17     semafaro
         17     site_ownership
         17     source:maxweight
         17     source:sccn
         17     source_name
         17     temp
         17     tipo
         17     trolley_wire:forward
         17     tunnel:length
         17     wheelchair:description:pt-BR
         17     wikipedia:es
         17     year_of_construction
         16     abandoned:loc_name
         16     abandoned:surface
         16     architect:wikidata
         16     bathing
         16     br:app
         16     brand:website
         16     check_date:fee
         16     cnpj
         16     commemorates:wikidata
         16     computer:repair
         16     fence
         16     forest
         16     golf:par
         16     guarded
         16     img_type
         16     lake
         16     lanes:both_ways
         16     language
         16     live_music
         16     mapillary:image
         16     motorcycle:clothes
         16     name:an
         16     name:arz
         16     name:be-tarask
         16     name:ie
         16     name:jv
         16     name:lij
         16     name:ln
         16     name:pms
         16     name:sa
         16     name:tk
         16     name:tl
         16     name:vo
         16     name:yue
         16     oneway:start_date
         16     operator:abbr
         16     parking:lane:left:perpendicular
         16     planned
         16     ref:CNS
         16     school:FR
         16     seamark:beacon_cardinal:category
         16     seamark:beacon_cardinal:colour
         16     seamark:beacon_cardinal:colour_pattern
         16     seamark:gate:category
         16     seamark:navigation_line:orientation
         16     seamark:wreck:water_level
         16     source:old_ref
         16     university
         16     unnecessary:addr:city
         16     whc:criteria
         15     ISO3166-1
         15     ISO3166-1:alpha2
         15     alt_source
         15     barrier:obstacle_type
         15     blind:description:pt-BR
         15     block
         15     buy
         15     community:gender
         15     currency:BTC
         15     currency:USDT
         15     disused:route
         15     disused:tourism
         15     door:wings
         15     emergency:social_facility
         15     est_height
         15     footway:crossing:markings
         15     free
         15     fuel:1_50
         15     fuel:octane_96
         15     gtfs:trip_id
         15     heading
         15     hotspot
         15     importance
         15     language:pt
         15     mall
         15     motor_vehicle:lanes:conditional
         15     name:bat-smg
         15     name:bo
         15     name:ckb
         15     name:ee
         15     name:ext
         15     name:frp
         15     name:gu
         15     name:hsb
         15     name:ht
         15     name:ilo
         15     name:jbo
         15     name:kl
         15     name:ky
         15     name:lmo
         15     name:mg
         15     name:nah
         15     name:pam
         15     name:so
         15     name:te
         15     name:tet
         15     name:vec
         15     name:wuu
         15     name:zu
         15     node
         15     parking:lane:side
         15     port
         15     raceway:corner_number
         15     ref:LOCODE
         15     safety_equipment
         15     school:type
         15     service:bicycle:bike
         15     silo
         15     sms
         15     trailblazed
         15     wpt_symbol
         15     xmas:feature
         14     Categoria
         14     FUNAI:REV
         14     IBGE:COD_BAIRRO
         14     ISO3166-1:alpha3
         14     ISO3166-1:numeric
         14     SME:CRE
         14     aerialway:access
         14     claimed_by
         14     cobblestone
         14     contact:pinterest
         14     diet:dairy_free
         14     diet:fruitarian
         14     diet:raw
         14     furniture
         14     generator:output:heat
         14     generator:output:hot_water
         14     generator:place
         14     genus:pt
         14     legenda
         14     monastery:type
         14     monitoring:precipitation
         14     motorcycle:tyres
         14     mtb
         14     mtb:description
         14     name:bcl
         14     name:bm
         14     name:dsb
         14     name:frr
         14     name:hif
         14     name:mhr
         14     name:nds
         14     name:pap
         14     name:sah
         14     note:highway
         14     official_name:ru
         14     old_name_1
         14     payment:amex
         14     population:municipality
         14     pos
         14     recreation_ground
         14     recycling:beverage_cartons
         14     recycling:cartons
         14     ref:BR:SNISB
         14     ref:bag
         14     ref:whc
         14     seamark:cable_submarine:name
         14     seamark:fortified_structure:category
         14     seamark:light:2:range
         14     seamark:light:sequence
         14     service:bicycle:mtb
         14     slogan
         14     source:amenity
         14     source:spelling
         14     speed
         14     unnecessary:addr:suburb
         14     was:name
         14     windows
         14     xmas:name
         13     Dimens├Áes
         13     access_aisle
         13     addr
         13     addr:source:housenumber
         13     alt_name:fr
         13     association
         13     bioma
         13     biome
         13     blood:plasma
         13     building:level
         13     building:name
         13     building:soft_storey
         13     cash_withdrawal
         13     check_date:lit
         13     clothing-optional
         13     codigo
         13     contact:postcode
         13     cycleway:left:lane
         13     diet:omnivore
         13     disabled
         13     disputed_by
         13     disused:landuse
         13     engineer
         13     guard_rail
         13     heating
         13     illuminated
         13     incorrect:leisure
         13     interval:conditional
         13     language:it
         13     language:ja
         13     level:ref
         13     lit_by_led
         13     monitoring:tide_scale
         13     name:ace
         13     name:als
         13     name:crh
         13     name:diq
         13     name:gag
         13     name:gv
         13     name:ltg
         13     name:nan
         13     name:nov
         13     name:rw
         13     name:szl
         13     name:vro
         13     name:wo
         13     name:xal
         13     official_name:fr
         13     parking:both:markings
         13     parking:lane:both:perpendicular
         13     parking:left:access
         13     population:district
         13     railway:switch:heated
         13     recycling:electronics
         13     recycling:fluorescent_tubes
         13     recycling:tyres
         13     roller_coaster
         13     sac_scale_ref
         13     seamark:crane:category
         13     size
         13     survey_point:purpose
         13     thong
         13     toll:motorcycle
         13     topless
         13     vehicle:lanes:backward
         13     vehicle:lanes:forward
         13     wikidata:fixme
         12     NIVEL_I
         12     NIVEL_II
         12     NIVEL_III
         12     abandoned:aeroway
         12     access:lanes:conditional
         12     agency_name
         12     codIBGE
         12     condephaat:inscription_date
         12     currency:ARS
         12     currency:LTC
         12     destination:colour:lanes
         12     diet:Carne
         12     diet:ovo_vegetarian
         12     entidad_te
         12     fuel:diesel_s500
         12     funicular
         12     handicap:red
         12     is_in:archipelago
         12     kml:guid
         12     lat
         12     lawyer
         12     light:power
         12     long
         12     mineshaft_type
         12     name:ang
         12     name:dz
         12     name:hak
         12     name:lez
         12     name:lo
         12     name:ps
         12     name:rn
         12     name:scn
         12     name:smn
         12     name:sms
         12     name:tzl
         12     name:wa
         12     name:wikipedia
         12     number_of_apartments
         12     n├¡veis
         12     official_name:cs
         12     opening_hours_1
         12     parking:lane:left:conditional
         12     recycling:PET
         12     recycling:books
         12     recycling:small_appliances
         12     riverbank
         12     seamark:control_point:category
         12     service:bicycle:clothing
         12     service:vehicle:injection
         12     service:vehicle:sale
         12     source:alt_ref
         12     source:leisure
         12     source:maxheight
         12     source:name:oc
         12     statue
         12     stream
         12     surface_1
         12     theme
         12     trolley_wire:description
         12     voltage-high
         12     was:shop
         12     whc:inscription_date
         12     wood:age
         11     abandoned:produce
         11     addr:postcode:right
         11     allotments
         11     altitude
         11     blood:stemcells
         11     contact:flickr
         11     curve
         11     cycleway:left:segregated
         11     cycleway:oneway
         11     disused:man_made
         11     disused:operator
         11     disused:waterway
         11     fee:conditional
         11     fonte
         11     format
         11     is_in:region
         11     lanes:psv:backward
         11     localizer
         11     massage
         11     museum_type
         11     name:bar
         11     name:bi
         11     name:cbk-zam
         11     name:ff
         11     name:km
         11     name:kv
         11     name:lad
         11     name:mrj
         11     name:nap
         11     name:rmy
         11     name:si
         11     name:zea
         11     nickname
         11     parking:left:fee:conditional
         11     passengers
         11     payment:litecoin
         11     payment:mercado_pago
         11     power_supply:voltage
         11     raised
         11     recycling:computers
         11     recycling:mobile_phones
         11     ref:UAI
         11     ref:wmo
         11     seamark:light:2:height
         11     service:bicycle:loja
         11     service:copy
         11     siren:type
         11     source:bridge
         11     stile
         11     store
         11     toilets:unisex
         11     traffic_signals:minimap
         11     treadmill
         11     valves
         11     vegetation
         11     wall:colour
         11     was:highway
         11     was:landuse
         10     AREA_HA
         10     Etapas de ensino
         10     Hor├írio de atendimento:
         10     abandoned:public_transport
         10     addr2:housenumber
         10     addr:postcode:left
         10     bike_ride
         10     bollard:count
         10     camp_site
         10     cep_impar
         10     climbing:boulder
         10     closest_town
         10     communication:lte+
         10     currency:USDC
         10     demolished:name
         10     disused:electrified
         10     disused:office
         10     disused:oneway
         10     drinking_water:legal
         10     embedded
         10     female:signed
         10     fenced
         10     flag:colour
         10     flag:wikipedia
         10     gambling
         10     give_way
         10     heritage:website
         10     hygrometer
         10     inline_skates
         10     label
         10     lanes:bus:forward:conditional
         10     male:signed
         10     marking
         10     mass_times
         10     maxspeed:hgv:forward
         10     monitoring:gps
         10     monitoring:rainfall
         10     mtb:name
         10     name:ha
         10     name:kab
         10     name:ki
         10     name:ks
         10     name:ksh
         10     name:mzn
         10     name:na
         10     name:nv
         10     name:srn
         10     name:xmf
         10     ne_id
         10     network_2
         10     not:landuse
         10     official_name:ca
         10     old_gauge
         10     old_operator:type
         10     opening_hours:drive_through
         10     parking:condition:left:default
         10     parking:left:maxstay:conditional
         10     payment:PIX
         10     payment:bancomat
         10     payment:transferencia
         10     plant:type
         10     pound
         10     prominence
         10     protection_object
         10     recycling:white_goods
         10     ref:cobacia
         10     ref:image
         10     restriction:bus
         10     sauna
         10     seamark:light:2:period
         10     seamark:light:2:sector_end
         10     seamark:light:2:sector_start
         10     service:bicycle:oficina
         10     sidewalk:both:tactile_paving
         10     sloped_curb
         10     socket:type2
         10     sound
         10     source:historic
         10     source:official_ref
         10     source:reg_name
         10     source:start_date
         10     swing_gate:type
         10     traffic_sign:2
         10     warehouse
         10     waterway_2
          9     IJSN:jurisdicao
          9     Padr├úo de implanta├º├úo
          9     abandoned:ref
          9     addr:street_1
          9     addr:village
          9     aerialway:heating
          9     antenna
          9     antenna:shape
          9     barometer
          9     building:start_date
          9     bunker_type
          9     cafe
          9     communication:bos
          9     cre
          9     currency:BCH
          9     currency:ETH
          9     default_language
          9     demolished:amenity
          9     depth:clearance
          9     description:de
          9     destination:ref:forward
          9     destroyed:building:levels
          9     diet:seafood
          9     diet:sugar_free
          9     display:digital
          9     easy_overtaking
          9     field
          9     floor:material
          9     fuel:e100
          9     fuel:octane_97
          9     gender_segregated
          9     grupo
          9     highway:2001-2002
          9     highway:lanes
          9     highway:lanes:forward
          9     horn
          9     incorrect_name
          9     int_ref
          9     kids_area:indoor
          9     maxaxleload
          9     motor_vehicle:lanes:backward:conditional
          9     name:bxr
          9     name:chr
          9     name:fur
          9     name:lg
          9     name:lzh
          9     name:rue
          9     name:sc
          9     name:simple
          9     name:ti
          9     name:to
          9     name_2
          9     name_alt
          9     network:note
          9     object:streetnumber
          9     official_name:de
          9     official_name:sv
          9     old_loc_name
          9     oldhousenumber
          9     open
          9     parking:lane:right:diagonal
          9     passenger
          9     payment
          9     payment:vale-alimenta├º├úo
          9     recycling:low_energy_bulbs
          9     recycling:tv_monitor
          9     ref:Aneel
          9     ref:CIAD
          9     ref:WDPA
          9     service:bicycle:ebike
          9     service:print
          9     service:vehicle:electronic_injection
          9     service:vehicle:paint_restoration
          9     share_taxi
          9     shooting
          9     sidewalk:both:kerb
          9     sidewalk:left:bicycle
          9     sidewalk_1
          9     source:prominence
          9     suburb
          9     tickets:public_transport
          9     wreck:visible_at_low_tide
          8     CD_GEOCODD
          8     DANE:area
          8     DANE:departamento
          8     Hor├írio de funcionamento
          8     IMPORTANCI
          8     Nome
          8     RODOVIA
          8     Uso atual da ├írea
          8     add
          8     addr:district:name
          8     addr:part
          8     alt_name:ca
          8     bicycle:backward
          8     blockage
          8     bridge:movable
          8     bus_stop:category
          8     cash_out
          8     change
          8     check_date:handrail
          8     colonnade:left
          8     communication:4G
          8     communication:Telephone
          8     communication:amateur_radio
          8     construction_date
          8     contact:tiktok
          8     crossing:activation
          8     currency:DASH
          8     cycle_barrier:installation
          8     denomination:wikimedia_commons
          8     destination:colour:forward
          8     destination:lang:es
          8     enclosed_area
          8     factory
          8     fair_trade
          8     fuel:diesel_s500_aditivado
          8     fuel:e27
          8     fuel:gasolina_C_comum
          8     fuel:oil
          8     gate
          8     gluten_free
          8     guide_type
          8     hgv:lanes:backward
          8     highway:2009-2010
          8     historic:operator
          8     historic_1
          8     ice_cream
          8     incorect:name
          8     interval:note
          8     ladder
          8     maxspeed:agricultural
          8     maxspeed:hgv:backward
          8     maxweightrating
          8     motorcycle:rental
          8     name:ab
          8     name:ak
          8     name:kaa
          8     name:kg
          8     name:krc
          8     name:mwl
          8     name:nrm
          8     name:pag
          8     name:pdc
          8     name:pih
          8     name:sg
          8     name:vep
          8     note1
          8     note:es
          8     official_name:eo
          8     official_name:id
          8     official_name:lt
          8     official_name:nl
          8     official_name:vi
          8     old_name1
          8     old_name:en
          8     old_surface
          8     oneway:foot
          8     opening_hours:source
          8     osm_id
          8     overtaking:forward
          8     parking:both:maxstay
          8     payment:banricompras
          8     payment:liquid
          8     payment:sodexo
          8     pentecostal
          8     photo
          8     plot_number
          8     public_bookcase:type
          8     raceway
          8     ref:FR:SIREN
          8     ref:INSEE
          8     roof:min_height
          8     school:BR
          8     scrub
          8     seamark:bridge:clearance_width
          8     seamark:buoy_safe_water:colour
          8     seamark:buoy_safe_water:colour_pattern
          8     seamark:buoy_safe_water:shape
          8     short_name:en
          8     socket:type2_cable
          8     source:barrier
          8     source:intermittent
          8     source:link
          8     source:name:en
          8     source:note
          8     source_type_walking_paper
          8     sports
          8     taxi:prefix:wheelchair
          8     toponimia
          8     tower
          8     traffic_signals:backward
          8     tripadvisor
          8     water_point
          8     website_1
          8     wpt_description
          8     xmas:day_date
          8     xmas:location
          7     2016olympicgames:cluster
          7     ADMINSTATE
          7     CBTC
          7     CD_GEOCODS
          7     Hor├írio de funcionamento:
          7     IPP:RAGIAO_ADM
          7     abandoned:landuse
          7     abutters
          7     access:lanes:forward
          7     addr:city:es
          7     addr:quarter
          7     addr:street:name
          7     aerialway:bubble
          7     alt_name2
          7     amenity_2
          7     animal_crossing
          7     area_calc_
          7     awash
          7     bakery
          7     barrier:personnel
          7     bridge:wikidata
          7     bus_routes
          7     butcher
          7     capacity:tents
          7     car
          7     carpenter
          7     check_date:bicycle
          7     checkpoint
          7     communication:3G
          7     communication:5g
          7     communication:military
          7     communication:umts
          7     communication:wifi
          7     currency
          7     cycleway:left:name
          7     defibrillator:location:pt
          7     demolished:official_name
          7     depth:source_quality
          7     depth:source_quality:reportyear
          7     destination:lanes:backward
          7     destination:lanes:forward
          7     destination:ref:backward
          7     diplomatic:services:immigrant_visas
          7     disused:barrier
          7     disused:emergency
          7     flower
          7     free_flying:paragliding
          7     fuel:aditivada
          7     fuel:lng
          7     healthcare:counselling
          7     highway:2003-2004
          7     highway:2003-2006
          7     imagery_used
          7     is_in:iso_3166_2
          7     lanes:cycleway:conditional
          7     language:others
          7     lhv
          7     lighthouse:note
          7     motor_vehicle:backward:conditional
          7     mountain_pass
          7     mown
          7     name:cdo
          7     name:eml
          7     name:mdf
          7     name:new
          7     name:sn
          7     name:stq
          7     official_name:be
          7     official_name:ckb
          7     official_name:ku
          7     official_name:sk
          7     official_name:zh
          7     parking:both:access
          7     parking:left:capacity
          7     parking:right:capacity
          7     parking:right:maxstay
          7     payment:food_voucher
          7     payment:free
          7     payment:prepaid_ticket
          7     payment:routex
          7     piste:difficulty
          7     psv:backward
          7     railway:milestone:emergency_brake_override
          7     railway:signal:direction
          7     razed:building
          7     recycling:shoes
          7     ref:category
          7     ref:colour
          7     ref:microred
          7     ref:red
          7     residential:units
          7     roof_shape
          7     self_checkout
          7     service:vehicle:car_sales
          7     service:vehicle:mechanics
          7     service:vehicle:repair
          7     service_1
          7     sidewalk:left:foot
          7     source:addr:interpolation
          7     source:bicycle
          7     source:leaf_type
          7     source:name:br
          7     spa
          7     supermarket
          7     tourism_1
          7     trainning
          7     uniPav
          7     vehicle:lanes
          7     zip_left
          6     2014worldcup
          6     DISTRITO
          6     Hydro_ID
          6     IPP:ORGAO2
          6     OBTC
          6     Testada
          6     access:delivery
          6     access:lanes:backward
          6     addr:housenumber2
          6     addr:housenumber_1
          6     addr:old_housenumber
          6     addr:town
          6     agency_number
          6     alt_name:de
          6     alt_name:eo
          6     alt_name:nl
          6     alt_namename
          6     asas
          6     barrier_1
          6     building:condition
          6     building:foundation_height
          6     building:parts
          6     building:roof:shape
          6     building:shape
          6     bus:colour
          6     busway:lane
          6     cash_withdrawal:foreign_cards
          6     cash_withdrawal:purchase_minimum
          6     changing_table:count
          6     check_date:bin
          6     climbing:trad
          6     clinic
          6     colonnade:right
          6     contact:spotify
          6     currency:CLP
          6     cycleway:right:separation
          6     cycleway:segregated
          6     dcgis:square
          6     defibrillator:location
          6     demolished
          6     depot:type
          6     description:pt_br
          6     designated
          6     destination:lang:en:lanes
          6     diplomatic:services:non-immigrant_visas
          6     disused:aerialway
          6     disused:brand
          6     disused:cuisine
          6     driveway
          6     emergency_telephone_code
          6     footprint
          6     fuel:diesel_G2
          6     fuel:natural_gas
          6     fuel:octane_93
          6     garden
          6     highway_1
          6     hospital
          6     hov:minimum
          6     ie
          6     impromptu
          6     insurance:health
          6     is_in:housename
          6     is_in:river
          6     kids_area:fee
          6     kids_area:supervised
          6     lane
          6     language:eo
          6     launche
          6     linen
          6     market
          6     maxheight:lanes
          6     maxlength:hgv
          6     maxstay:conditional
          6     maxwidth:physical
          6     medical_system:western
          6     name:arc
          6     name:etymology
          6     name:gan
          6     name:kbd
          6     name:pcd
          6     name:roa-tara
          6     name:sm
          6     name:ss
          6     name:vls
          6     name:wap
          6     name:way
          6     name:za
          6     ne:brk_a3
          6     neighbourhood
          6     network_3
          6     not:IBGE:CD_ADMINIS
          6     number
          6     official_name:ar
          6     official_name:el
          6     official_name:eu
          6     official_name:it
          6     official_name:lb
          6     official_name:lv
          6     official_name:pl
          6     official_name:tr
          6     oneway:psv
          6     openplaques:id
          6     operation
          6     osmc:symbol
          6     parking:both:charge
          6     parking:condition:right:capacity
          6     parking:condition:right:except
          6     payment:bankaxess
          6     payment:laser
          6     payment:meal_voucher
          6     payment:postfinance_card
          6     personal_trainer
          6     place:destination
          6     place:origin
          6     port:type
          6     post
          6     proposed:railway
          6     proposed:service
          6     qualidade
          6     ref:CNPJ
          6     ref_1
          6     relation
          6     review
          6     right:8
          6     ruined:building
          6     sand
          6     seamark:light:1:sequence
          6     seamark:obstruction:water_level
          6     seamark:radar_transponder:range
          6     security
          6     service:bicycle:stand
          6     service:prepress
          6     service:vehicle:car_service
          6     short_name:ru
          6     shoulder:width
          6     shrubbery:density
          6     site_name
          6     sou
          6     source:merge
          6     source:short_name
          6     source:website
          6     source_2
          6     species:es
          6     stage
          6     subdistrict
          6     summit:cross
          6     technology
          6     temporary:access
          6     toilets:paper_supplied
          6     twitter
          6     valve
          6     veterinary
          6     volcano:status
          6     wall:material
          6     water_tank:volume
          6     wreck:visible_at_high_tide
          5     Condi├º├úo atual da ├írea
          5     ID
          5     IJSN:gestao
          5     Linetype
          5     NOM_FANTAS
          5     Rio
          5     abandoned:leisure
          5     abandoned:service
          5     access:hgv
          5     access_control
          5     addr:community
          5     addr:municipality
          5     addr:streetnumber
          5     aerialway:duration
          5     apartments
          5     avgspeed
          5     bank
          5     beach
          5     bell_tower
          5     blind:description:pt
          5     blood:blood_donation
          5     board:title
          5     booth
          5     building:use:residential
          5     bus:backward
          5     busway:both
          5     camera:angle
          5     car_parts
          5     cargo:general
          5     carriage
          5     cash_withdrawal:fee
          5     cash_withdrawal:type
          5     check_date:compressed_air
          5     check_date:wheelchair
          5     climbing:multipitch
          5     communication:wcdma
          5     concrete
          5     construction:amenity
          5     contact:tripadvisor
          5     cooler
          5     currency:XMR
          5     cyclability
          5     cycleway:right:conditional
          5     demolished:man_made
          5     description:es
          5     destination:arrow:lanes
          5     destination:colour:backward
          5     destination:es
          5     destination:pt
          5     destination:street:forward
          5     destination:symbol:lanes
          5     destroyed:height
          5     distillery
          5     disused:ref
          5     drink:water
          5     ethanol
          5     fclass
          5     ferry:cable
          5     fix
          5     flats
          5     flickr
          5     floodplain
          5     floor:material:type
          5     flow_direction
          5     former_product
          5     fuel:Arla_32
          5     fuel:e20
          5     fuel:gasolina_podium
          5     gated_community
          5     hazard:animal
          5     health_amenity:type
          5     highway:2002-2003
          5     honk
          5     hot_tub
          5     indoor:area
          5     inepac:inscription_date
          5     isced:level:1997
          5     kids_area:outdoor
          5     length_unit
          5     lg_nome
          5     lg_tipo
          5     maxspeed:forward:conditional
          5     name:as
          5     name:azb
          5     name:botanical
          5     name:csb
          5     name:cu
          5     name:description
          5     name:haw
          5     name:ig
          5     name:min
          5     name:nds-nl
          5     name:nso
          5     name:om
          5     name:pfl
          5     name:tok
          5     name:tpi
          5     name:ty
          5     name:udm
          5     nightclub
          5     note:de
          5     note:lanes
          5     official_name:br
          5     official_name:et
          5     official_name:fa
          5     official_name:gn
          5     official_name:ja
          5     official_name:ro
          5     official_name:sl
          5     official_name:sr
          5     official_name:tl
          5     official_name:ur
          5     official_name:zh-Hans
          5     official_name:zh-Hant
          5     old_phone
          5     oneway:source
          5     opening_hours:kitchen
          5     operator_1
          5     panoramax:2
          5     parcel_pickup
          5     parent_basin
          5     parking:condition:both:default
          5     parking:left:surface
          5     pathtype
          5     paving_stones:colour
          5     payment:alelo_refei├º├úo
          5     payment:card
          5     payment:pagseguro
          5     payment:samsung_pay
          5     payment:tag
          5     payment:ticket_alimenta├º├úo
          5     payment:wire_transfer
          5     pond_use
          5     post_code
          5     protect_id
          5     pub
          5     recycling:household_appliances
          5     ref:BR:CIAD
          5     ref:MEC
          5     ref:sisa_tipologia
          5     research
          5     roof:ridge
          5     roundabout
          5     safety_inspection
          5     seamark:dumping_ground:category
          5     seamark:light:3:character
          5     seamark:light:3:colour
          5     seamark:light:3:height
          5     seamark:seabed_area:minimum_depth
          5     seats:separated
          5     service:bicycle:screwdriver
          5     service:vehicle:muffler
          5     short_name:cs
          5     short_name:de
          5     short_protection_title
          5     sidewalk:both:foot
          5     sidewalk:both:name
          5     sidewalk:both:width
          5     site_status
          5     source:addr:postcode
          5     source:bridge:name
          5     source:contact
          5     source:imagery
          5     source:junction
          5     source:overtaking
          5     source_3
          5     street
          5     surface:source
          5     survey
          5     tables
          5     theatre
          5     traffic_calming:right
          5     transport
          5     underwear
          5     unpaved
          5     was:maxspeed
          5     weather:barometer
          5     weather:thermometer
          5     width:sidewalk
          5     wikipedia:ru
          5     wreck:date_sunk
          5     xmas:opening_hours
          5     yacht
          4     IPP:CARACTERIS
          4     IPP:TUTELA2
          4     LEI
          4     Note
          4     Padr├úo de instala├º├úo
          4     REF
          4     Tag
          4     abandoned:lanes
          4     abandoned:place
          4     abandoned:shop
          4     abandoned:waterway
          4     access:disabled
          4     access:note
          4     addr:level
          4     adm
          4     admin_leve
          4     alt_name:1
          4     alt_name:br
          4     alt_name:cs
          4     alt_name:gl
          4     alt_name_3
          4     barrier_type
          4     bench:type
          4     bicycle:forward
          4     bicycle:surface
          4     bing
          4     border
          4     bridge:alt_name
          4     bridge:length
          4     bridge:loc_name
          4     bus:forward
          4     camping:tents
          4     campo
          4     capacity:caravans
          4     capacity:senior
          4     caravan
          4     cash_withdrawal:currency
          4     change:lanes
          4     changeset
          4     child
          4     city_limit
          4     climbing:length
          4     climbing:toprope
          4     clock
          4     clothes:rental
          4     communication:4g+
          4     communication:ham_radio
          4     communication:operator
          4     communication:radio_internet
          4     community:br
          4     community:pt
          4     confession_times
          4     construction:power
          4     contact:centrex
          4     contact:pobox
          4     currency:PYG
          4     cycle_barrier
          4     cyclestreet
          4     cycleway:official_name
          4     cycleway:right:foot
          4     cycleway:right:width
          4     cycleway:short_name
          4     cycling
          4     danger
          4     day_off
          4     demolished:bridge
          4     departures
          4     destination:ref:lanes
          4     destroyed:surface
          4     diesel
          4     diet:egg_free
          4     diet:mediterranean
          4     disused:area
          4     disused:attraction
          4     disused:bus
          4     disused:industrial
          4     disused:lamp_mount
          4     disused:lamp_type
          4     drink
          4     drink:coconut_water
          4     drink:cola
          4     drink:juice
          4     drink:soft_drink
          4     drink:tea
          4     email_1
          4     escalator
          4     fence:type
          4     fire_hydrant:count
          4     fish
          4     fixme:access
          4     free_flying:hanggliding
          4     function
          4     garage
          4     gas_insulated
          4     goods:conditional
          4     green
          4     hazard_intensity
          4     heath
          4     highway:2001-2003
          4     highway:2003-2007
          4     highway:2005-2006
          4     highway:2007-2008
          4     highway:conditional
          4     hinge
          4     incorrect:tourism
          4     info
          4     isced:level:2011
          4     iucn_level
          4     karaoke
          4     kerb:approach_aid
          4     lamp_model
          4     land_area
          4     lanes:forward:conditional
          4     lanes:psv:right
          4     library
          4     light:flux
          4     link
          4     loc_name:en
          4     markings
          4     maxspeed:bus:backward
          4     maxspeed:lanes:conditional
          4     megalith_type
          4     milestone
          4     mkgmap:area2poi
          4     monitoring:meteorology
          4     monitoring:tide
          4     motorcycle:type
          4     name1
          4     name:av
          4     name:bh
          4     name:language
          4     name:nb
          4     name:prefix
          4     name:st
          4     name:ve
          4     nota
          4     note:maxspeed
          4     objectid
          4     official_name:an
          4     official_name:azb
          4     official_name:chr
          4     official_name:fi
          4     official_name:gl
          4     official_name:he
          4     official_name:hr
          4     official_name:hu
          4     official_name:ko
          4     official_name:nds
          4     official_name:no
          4     official_name:oc
          4     official_name:qu
          4     official_name:sco
          4     official_name:sq
          4     official_name:su
          4     official_name:ta
          4     official_name:uz
          4     offloading
          4     old_name:vi
          4     old_ref_2
          4     one
          4     oneway:motor_vehicle
          4     opening_hours:emergency
          4     opening_hours:note
          4     opera
          4     operator:wikimedia_commons
          4     origem
          4     outlet
          4     owner:wikipedia
          4     parking:both:parallel
          4     parking:both:restriction:conditional
          4     parking:both:restriction:motorcycle:conditional
          4     parking:condition:left:fee_per_hour
          4     parking:condition:right:fee_per_hour
          4     parking:lane:right:perpendicular
          4     parking:left:authentication:ticket:conditional
          4     parking:left:charge
          4     parking:left:maxstay
          4     parking:right:maxstay:conditional
          4     parking:right:restriction:conditional
          4     payment:bitcoincash
          4     payment:conectcar
          4     payment:ep_avant
          4     payment:ep_minipay
          4     payment:sicredi
          4     payment:ticket
          4     place_of_worship
          4     pond
          4     post:housenumber
          4     postal
          4     railway:disused
          4     railway_car
          4     ramsar
          4     recycling:drugs
          4     recycling:furniture
          4     recycling:lamps
          4     recycling:small_electrical_appliances
          4     ref:FR:SIRET
          4     ref:Prefeitura_de_Contagem
          4     ref:UY:snapid
          4     ref:isil
          4     ref:itu
          4     ref:sernap
          4     region:type
          4     reservoir
          4     restriction:reason
          4     road
          4     scooter
          4     seamark:cable_overhead:clearance_height_safe
          4     seamark:calling-in_point:callsign
          4     seamark:ferry_route:category
          4     seamark:light:3:range
          4     seamark:light:3:sector_end
          4     seamark:light:3:sector_start
          4     service:bicycle:dealer
          4     service:bicycle:wheel
          4     service:vehicle:alarm
          4     service:vehicle:detailing
          4     service:vehicle:engine_repair
          4     service:vehicle:exhausts
          4     service:vehicle:new_truck_sales
          4     service:vehicle:tyres_repair
          4     shop_2
          4     short_name:es
          4     short_name:fr
          4     short_name:nl
          4     short_name:pt
          4     small_electric_vehicle
          4     source:bridge_name
          4     source:bus
          4     source:cycleway:width
          4     source:phone
          4     source:stream
          4     source:traffic_sign
          4     stateofrepair
          4     summit_area
          4     sur
          4     surf
          4     swimming
          4     sym
          4     teaching
          4     telescope:type
          4     temperature
          4     toll:bicycle
          4     toll:psv
          4     was:leisure
          4     was:operator
          4     was:website
          4     weather:anemometer
          4     website:orders
          4     windings
          3     Acesso
          3     Authority
          3     Bing
          3     DGPS_correction
          3     FUNAI:OBS_AREA
          3     NM_METRICA
          3     NOMBRE
          3     NOM_TIPOLO
          3     Shape_Area
          3     Terreno
          3     abandoned:nat_ref
          3     abandoned:tourism
          3     abbr
          3     accommodation
          3     addr:city_block
          3     addr:housenumber:source
          3     addr:locality
          3     addr:loja
          3     addr:plot
          3     addr:postcode:source
          3     addr:quadra
          3     adress
          3     aerialway:summer:access
          3     aircraft:type
          3     alt_name:ar
          3     alt_name:it
          3     alt_name:vi
          3     altitude_high
          3     altitude_low
          3     ambulance
          3     anchoring
          3     animal_shelter:adoption
          3     animal_training
          3     artificial
          3     author
          3     balcony
          3     barrier:colour
          3     bicycle:lanes
          3     bicycle:lanes:conditional
          3     bicycle:signed
          3     brand_1
          3     brothel
          3     building:levels_1
          3     building:min_levels
          3     building:ruian:type
          3     bus_bay:right
          3     capital_city
          3     car_repair
          3     cargo:dry_bulk
          3     cash_withdrawal:operator
          3     cec:inscription_date
          3     cell_size
          3     check_date:kerb
          3     check_date:sidewalk
          3     check_date:traffic_signals:sound
          3     clcfile
          3     coastline:surveyquality
          3     collection
          3     comment:pt_BR
          3     communication:VHF
          3     communication:aeronautical_mobile_service
          3     communication:amateur_radio:repeater
          3     communication:isp
          3     communication:tv
          3     constructed
          3     construction:building:levels
          3     construction_end_expected
          3     construction_start_expected
          3     contact:city
          3     contact:country
          3     contact:housenumber
          3     contact:place
          3     contact:street
          3     controlled_by
          3     country_code_fips
          3     cycleway:lane
          3     cycleway:right:name
          3     cycleway:smoothness
          3     day_on
          3     demolished:railway
          3     demolished:source:geometry
          3     destination:to:backward
          3     diet:carnivore
          3     diet:churrasco
          3     diet:diabetes
          3     direction:lanes
          3     dirindicator
          3     disused:craft
          3     disused:depot
          3     disused:phone
          3     disused:sport
          3     disused:tunnel
          3     end_time
          3     ep
          3     events_venue
          3     fell
          3     fixme:icao
          3     flow_control
          3     flowers
          3     footway:levels
          3     format:top
          3     free_flying:official
          3     fruit
          3     fuel:gasohol_91
          3     fuel:regular
          3     glass
          3     glideslope
          3     gps
          3     greenfield
          3     gym
          3     gymnasium
          3     handle
          3     health_specialty:ophthalmology
          3     hgv:forward
          3     hgv_articulated
          3     highway:1999-2009
          3     highway:2002-2005
          3     highway:2008-2009
          3     highway:class:pl
          3     highway:disused
          3     historic:amenity
          3     historic:electrified
          3     historic:gauge
          3     historic:name
          3     historic:usage
          3     indoor:door
          3     is_in:department
          3     isced
          3     kitesurf
          3     lanes:extra
          3     language:ch
          3     language:zh
          3     leisure_1
          3     leisure_4
          3     lines
          3     lutheran
          3     maintained
          3     marketplace
          3     maxspeed:hgv:conditional
          3     meat
          3     monitoring:bicycle
          3     monitoring:telecomunication
          3     monitoring:water_quality
          3     motorhome
          3     motorway
          3     name:ady
          3     name:alt
          3     name:ch
          3     name:chy
          3     name:dty
          3     name:fil
          3     name:fj
          3     name:gcf
          3     name:gom
          3     name:ii
          3     name:jam
          3     name:kbp
          3     name:kk-Arab
          3     name:lfn
          3     name:lrc
          3     name:mai
          3     name:myv
          3     name:olo
          3     name:pi
          3     name:tw
          3     name:xh
          3     naturbase:iid
          3     nm_lote
          3     nm_quadra
          3     not:width
          3     notas
          3     note:shoulder
          3     notes
          3     obelisk:size
          3     official_name:af
          3     official_name:am
          3     official_name:ast
          3     official_name:ay
          3     official_name:ba
          3     official_name:be-tarask
          3     official_name:bg
          3     official_name:crh
          3     official_name:cy
          3     official_name:da
          3     official_name:dsb
          3     official_name:ext
          3     official_name:frr
          3     official_name:gu
          3     official_name:hi
          3     official_name:hsb
          3     official_name:hy
          3     official_name:ia
          3     official_name:ilo
          3     official_name:is
          3     official_name:ka
          3     official_name:kk
          3     official_name:krc
          3     official_name:lez
          3     official_name:li
          3     official_name:lo
          3     official_name:lzh
          3     official_name:mi
          3     official_name:mk
          3     official_name:mn
          3     official_name:mt
          3     official_name:nah
          3     official_name:pa
          3     official_name:pfl
          3     official_name:ps
          3     official_name:se
          3     official_name:sh
          3     official_name:si
          3     official_name:te
          3     official_name:th
          3     official_name:tk
          3     official_name:tt
          3     official_name:ug
          3     official_name:uk
          3     official_name:vep
          3     official_name:vls
          3     official_name:vro
          3     official_name:war
          3     official_name:xal
          3     official_name:yi
          3     official_name:yo
          3     official_name:yue
          3     old_name:1925
          3     old_official_name
          3     oneway:motorcar
          3     oneway:taxi
          3     onw
          3     openfire
          3     opening_hours_2
          3     operator:email
          3     operator_type
          3     overtaking:backward
          3     parking:both:capacity
          3     parking:condition:both:conditional
          3     parking:condition:both:vehicles
          3     parking:condition:left:capacity
          3     parking:condition:left:except
          3     parking:lane:left:diagonal
          3     parking:left:motorcycle
          3     parking:left:restriction:hgv
          3     parking:left:restriction:motorcycle
          3     parking:right:charge
          3     pastry_shop
          3     paving
          3     paving_stones:material
          3     payment:banricard
          3     payment:bilhete_├║nico
          3     payment:boleto_Bancario
          3     payment:credsystem
          3     payment:money_order
          3     payment:qr_code
          3     payment:sorocred
          3     payment:unionpay
          3     payment:visa_contactless
          3     pe:minedu:codigo_local
          3     phone2
          3     phone:BR
          3     phone_2
          3     picnic_table
          3     place:suburb
          3     platform_lift
          3     plots
          3     port_of_entry
          3     postal_code_level
          3     presence
          3     proposed:aerialway
          3     proposed:name
          3     public_building
          3     radio_transponder:category
          3     railway:abandoned
          3     railway:razed
          3     railway:service
          3     railway:signal:whistle:form
          3     railway:switch:movable_frog
          3     real_ale
          3     recognized_by
          3     recording:remote
          3     recycling:construction_waste
          3     recycling:light_bulbs
          3     ref:FR:FANTOIR
          3     ref:FR:LaPoste
          3     ref:sisa_refes
          3     residential:type_idn
          3     ruins:building:levels
          3     seamark:daymark:colour
          3     seamark:daymark:shape
          3     seamark:landmark:conspicuity
          3     seamark:landmark:name
          3     seamark:light:2:group
          3     seamark:light:3:period
          3     seamark:radio_station:frequency
          3     seamark:restricted_area:name
          3     seamark:virtual_aton:category
          3     sector:name
          3     service:bicycle:vending
          3     service:vehicle
          3     service:vehicle:battery_recharge
          3     service:vehicle:battery_testing
          3     service:vehicle:truck_parts
          3     service:vehicle:truck_sales
          3     service:vehicle:used_truck_sales
          3     ship:type
          3     shoes:repair
          3     short_name:pl
          3     short_name:sv
          3     short_name:uk
          3     sidewalk:right:segregated
          3     socket:type2_combo
          3     soft_storey
          3     source:addr:housename
          3     source:geo
          3     source:geometry:version
          3     source:hazard
          3     source:housename
          3     source:natural
          3     source:noexit
          3     source:shop
          3     source:wheelchair
          3     start_time
          3     step:height
          3     steps:height
          3     straightened
          3     tailor:alteration_service
          3     taxon:en
          3     television
          3     traffic_calming_1
          3     trailer
          3     truck:repair
          3     turn:backward
          3     unnecessary:motor_vehicle
          3     urban
          3     vhf
          3     viaduct
          3     volcano:type
          3     was:man_made
          3     was:religion
          3     was:resource
          3     was:start_date
          3     was:tourism
          3     water_tower
          3     website:regulation
          3     whatsapp
          3     wheelchair:description:pt
          3     wikipedia:gl
          3     windsurf
          3     workshop
          3     year
          3     youtube
          3     zip_right
          2     3dr:direction
          2     3dr:length1
          2     AREA
          2     CEP_LD
          2     CEP_LE
          2     ESTADO
          2     Escola
          2     Fonte
          2     Gin├ísio
          2     HECTARES
          2     Hor├írio de atendimento
          2     IBGE
          2     IFR
          2     IPP:ORGAO3
          2     Neighborhood
          2     Observation
          2     PERIMETER
          2     Quadra
          2     Tabatinga-central
          2     Telado
          2     abandoned:brand
          2     abandoned:bridge
          2     abandoned:power
          2     abbr_name_1
          2     access:gender
          2     addr2:suburb
          2     addr:city_1
          2     addr:conscriptionnumber
          2     addr:lote
          2     addr:name
          2     addr:neighborhood
          2     addr:sector
          2     addr:street:sym_ul
          2     admin
          2     administrative
          2     aeroway:gate:type
          2     air_rescue_service
          2     alt_name:gd
          2     alt_name:gn
          2     alt_name:hi
          2     alt_name:mzn
          2     alt_name:oc
          2     alt_name:ta
          2     alt_name_4
          2     alt_phone
          2     alt_shop
          2     amenity_3
          2     amount
          2     animals
          2     apartment
          2     architect:wikipedia
          2     arid
          2     asphalt
          2     atm:network
          2     atm:operator
          2     attribution
          2     avg
          2     bases
          2     bay
          2     beef
          2     bell:multiple
          2     bicycle:foldable
          2     bike
          2     billiards
          2     billiards:snooker
          2     blood_donation
          2     booking
          2     breakfast:fee
          2     bridge:name:en
          2     building:facade:colour
          2     building:levels:aboveground
          2     building:material:concrete
          2     building:min_height
          2     building:part_1
          2     building:roof:levels
          2     building:use:retail
          2     buildingpart:verticalpassage
          2     bulk_purchase
          2     camera
          2     camping
          2     camping:parts
          2     canoe_rental
          2     cap
          2     car:repair
          2     cargo_bike
          2     cattle
          2     charging_station:output
          2     check_date:crossing:barrier
          2     check_date:ramp
          2     check_date:tracktype
          2     circular
          2     clothes:women
          2     coat_of_arms
          2     code
          2     college
          2     colour_1
          2     colour_2
          2     communication:5G
          2     communication:amateur_radio:repeater:frequency_out
          2     communication:aviation
          2     computer:parts
          2     conditional
          2     construction:building
          2     construction:public_transport
          2     construction:railway:cbtc
          2     construction_year
          2     contact:deezer
          2     contact:foursquare
          2     contact:google_plus
          2     contact:mastodon
          2     contact:phone:1
          2     contact:tumblr
          2     contact:unit
          2     contact:vimeo
          2     contact:website:menu
          2     count
          2     course
          2     crater:age
          2     credit_cards
          2     crematorium
          2     crop_1
          2     crossing:on_demand
          2     cuisine:ja
          2     currency:BNB
          2     currency:CAD
          2     currency:GBP
          2     currency:JPY
          2     currency:PEN
          2     cycleway:crossing
          2     cycleway:left:surface
          2     def:opening_hours:PH
          2     demolished:height
          2     denomination_1
          2     depth:swept
          2     desc
          2     description:covid19
          2     description:ja
          2     destination:lang:eu
          2     destination:name
          2     destination:to:forward
          2     destroyed:bridge:structure
          2     destroyed:lanes
          2     destroyed:layer
          2     diet:lowcarb
          2     diet:mineira
          2     diet:salgados
          2     diplomatic:services:consulate
          2     diplomatic:services:registry_office
          2     diplomatic:services:visa
          2     direction:backward
          2     disused:admin_level
          2     disused:boundary
          2     disused:brand:wikidata
          2     disused:crossing
          2     disused:crossing:markings
          2     disused:entrance
          2     disused:fee
          2     disused:ferry
          2     disused:government
          2     disused:healthcare
          2     disused:internet_access
          2     disused:junction
          2     disused:name_1
          2     disused:parking
          2     disused:place
          2     disused:police
          2     disused:studio
          2     disused:toll
          2     dressing_room
          2     drink:soda
          2     earthquake
          2     earthquake:damage
          2     education_fee
          2     education_level:preschool
          2     education_level:primary
          2     ele:ellipsoid
          2     ele:ft
          2     ele:note
          2     elec:frequency
          2     elec:voltage
          2     endlevel
          2     equestrian
          2     etnia_nome
          2     events
          2     exercise
          2     farm
          2     fashion
          2     favela
          2     fitness
          2     fitness_centre
          2     fixed
          2     fixme:housenumber
          2     fixme:phone
          2     flag:country
          2     flat_steps
          2     floors
          2     free_flying
          2     free_flying:rigid
          2     fuel:anidro
          2     fuel:diesel_B
          2     fuel:discount
          2     fuel:e25
          2     fuel:gas
          2     fuel:gasoil
          2     fuel:gasoline_95
          2     fuel:petrol
          2     fuel:vpower
          2     f├íbrica
          2     gate:colour
          2     generator:output:biogas
          2     geoglyph
          2     geographic
          2     geometry_s
          2     geonames:id
          2     grave
          2     group
          2     guardrail
          2     hail_and_ride
          2     harbour:operator
          2     health_specialty:cardiology
          2     health_specialty:dentistry
          2     health_specialty:physiotherapy
          2     healthcare:speciality:de
          2     hearing impaired
          2     hei
          2     helipad
          2     heritage:since
          2     hifi:repair
          2     highway:1985-1988
          2     highway:1999-2001
          2     highway:1999-2006
          2     highway:1999-2007
          2     highway:1999-2011
          2     highway:2002-2006
          2     highway:2002-2007
          2     highway:lanes:backward
          2     historic:building
          2     historic:man_made
          2     historic:owner
          2     historic:period
          2     horse:conditional
          2     hostel
          2     hotel
          2     id:cnpj
          2     id:fetranspor
          2     id:ie
          2     id_des
          2     id_spn
          2     id_via_int
          2     inauguration
          2     indefinite
          2     industry
          2     international_flights
          2     internet_access:provider
          2     internet_access:type
          2     intersection
          2     is_in:hamlet
          2     is_in:postcode
          2     jetski
          2     kartaview
          2     keepright
          2     key
          2     kids
          2     landfill:waste
          2     lanes:taxi:forward
          2     lbcs:activity:name
          2     leisure_2
          2     leisure_3
          2     lengths:left
          2     level_crossing
          2     livestock_dip
          2     local
          2     local_authority:FR
          2     locator
          2     lot_type
          2     lottery
          2     magic_wand_merge
          2     manmade
          2     maxheight:backward
          2     maxheight:forward
          2     maxweight:hgv:conditional
          2     mead
          2     memorial:type
          2     minspeed:lanes:backward
          2     modalidade
          2     monitoring:air_traffic
          2     monitoring:glonass
          2     monitoring:radiation
          2     monitoring:wind_speed
          2     motor_vehicle:backward
          2     music
          2     name:2
          2     name:aa
          2     name:bjn
          2     name:bridge
          2     name:brx
          2     name:bug
          2     name:cnr
          2     name:gsw
          2     name:ja-Hira
          2     name:ja-Latn
          2     name:koi
          2     name:official
          2     name:pnt
          2     name:pt_BR
          2     name:roa-rup
          2     name:rom
          2     name:sd
          2     name:sr-Latn
          2     name:ts
          2     name:tum
          2     name:tyv
          2     name:zh-Hant-CN
          2     name:zh-Hant-HK
          2     name:zh-Hant-TW
          2     name_3
          2     navigable
          2     ngo
          2     non_existent_levels
          2     not:brand
          2     not:destination
          2     not:destination:street
          2     not:motor_vehicle
          2     note:population
          2     note:route
          2     note:url
          2     note_1
          2     object:housenumber
          2     official_name:als
          2     official_name:arz
          2     official_name:az
          2     official_name:bm
          2     official_name:bn
          2     official_name:bpy
          2     official_name:bs
          2     official_name:bxr
          2     official_name:cbk-zam
          2     official_name:ce
          2     official_name:cv
          2     official_name:fo
          2     official_name:fy
          2     official_name:gan
          2     official_name:gd
          2     official_name:gv
          2     official_name:ha
          2     official_name:io
          2     official_name:jam
          2     official_name:kbd
          2     official_name:kl
          2     official_name:kn
          2     official_name:kv
          2     official_name:ky
          2     official_name:la
          2     official_name:lad
          2     official_name:lmo
          2     official_name:ltg
          2     official_name:mai
          2     official_name:mg
          2     official_name:ml
          2     official_name:ms
          2     official_name:mwl
          2     official_name:my
          2     official_name:mzn
          2     official_name:nan
          2     official_name:nap
          2     official_name:nds-nl
          2     official_name:ne
          2     official_name:nv
          2     official_name:olo
          2     official_name:os
          2     official_name:pam
          2     official_name:pap
          2     official_name:pms
          2     official_name:rm
          2     official_name:rmy
          2     official_name:roa-tara
          2     official_name:rue
          2     official_name:sah
          2     official_name:sm
          2     official_name:sw
          2     official_name:tet
          2     official_name:vec
          2     official_name:wo
          2     official_name:wuu
          2     official_name:xmf
          2     official_status
          2     old:shop
          2     old_addr:housename
          2     old_description
          2     old_name:1920
          2     old_name:2013-2015
          2     old_name:2019-2023
          2     old_name:fr
          2     old_name:pt
          2     old_name:ru
          2     old_operator:2006
          2     old_operator:short
          2     old_short_name
          2     on_air:frequency
          2     oneway:2015-10-15--
          2     oneway:2015-10-16--
          2     oneway:2015-10-20--
          2     oneway:2015-10-21--
          2     oneway_1
          2     opening_hours:atm
          2     opening_hours:delivery
          2     opening_hours:reception
          2     operating_hours
          2     oven
          2     owner:short
          2     parcel_mail_in
          2     parking:both:hgv
          2     parking:condition:both:fee_per_hour
          2     parking:condition:default
          2     parking:condition:right:conditional
          2     parking:condition:time_interval
          2     parking:lane:both:capacity
          2     parking:lane:both:diagonal
          2     parking:lane:left:capacity:disabled
          2     parking:lane:right:capacity:disabled
          2     parking:left:capacity:disabled
          2     parking:left:charge:conditional
          2     parking:left:restriction:motorcycle:conditional
          2     parking:right:hgv
          2     parking:right:motorcycle
          2     parking_1
          2     parque
          2     particular
          2     party
          2     passenger_information_display:speech_output
          2     passing_place
          2     pastor
          2     pastry
          2     patron_saint
          2     payment:AME_Digital
          2     payment:a_vista
          2     payment:airbnb
          2     payment:alimenta├º├úo
          2     payment:all
          2     payment:android_pay
          2     payment:bank_slip
          2     payment:credit
          2     payment:ep_mep
          2     payment:ep_minicash
          2     payment:ep_monedero
          2     payment:ep_monedero4b
          2     payment:ethereum
          2     payment:hiper
          2     payment:movemais
          2     payment:nfc_mobile_payments
          2     payment:phone
          2     payment:stripe
          2     payment:sus
          2     payment:ticket_restaurant
          2     payment:toll_tag
          2     payment:verdecard
          2     payment:vr_alimenta├º├úo
          2     pharmacy
          2     phone1
          2     phone:UA
          2     phone_alt
          2     photography
          2     piste:grooming
          2     placement:forward
          2     plant_1
          2     plants
          2     pmasefin:idedif
          2     police:FR
          2     political_division
          2     polling_place
          2     post:street
          2     post_office
          2     post_office:type
          2     postal_code_1
          2     postal_code_2
          2     press
          2     price
          2     priority
          2     proposed:building
          2     proposed:generator:output:electricity
          2     proposed:generator:source
          2     proposed:leisure
          2     proposed:pitch
          2     proposed:power
          2     proposed:public_transport
          2     protect_area
          2     protect_title
          2     public_transport:disused
          2     public_transport_1
          2     quality
          2     radar
          2     railway:local_operated:lockable
          2     railway:signal:position
          2     real_fire
          2     recycling:aluminum
          2     recycling:cds
          2     recycling:chipboard
          2     recycling:entulho
          2     recycling:iron
          2     recycling:mattresses
          2     recycling:organic_waste
          2     recycling:polystyrene_foam
          2     recycling:printer_cartridges
          2     recycling:tetrapak
          2     red_turn:right
          2     ref:FR:GendarmerieNationale
          2     ref:admiralty
          2     ref:anac
          2     ref:detran
          2     ref:nga
          2     ref:nivel
          2     ref:ruian:addr
          2     ref:sniic
          2     ref_2
          2     reference_point
          2     refitted
          2     regionname
          2     regulation
          2     removed:highway
          2     removed:oneway
          2     req
          2     retail
          2     reusable_packaging:accept
          2     roadtrain
          2     roof:edge
          2     roof:material:type
          2     ruined
          2     ruins:historic
          2     ruins:man_made
          2     rural
          2     safety:mask:covid19
          2     safety_rope
          2     sample_collection
          2     school:language
          2     scuba_diving:divespot
          2     seamark:buoy_installation:category
          2     seamark:landmark:height
          2     seamark:light:1:category
          2     seamark:light:4:character
          2     seamark:light:4:colour
          2     seamark:light:4:height
          2     seamark:light:4:range
          2     seamark:light:4:sector_end
          2     seamark:light:4:sector_start
          2     seamark:light:5:character
          2     seamark:light:5:colour
          2     seamark:light:5:height
          2     seamark:light:5:range
          2     seamark:light:5:sector_end
          2     seamark:light:5:sector_start
          2     seamark:mooring:colour
          2     seamark:obstruction:depth
          2     seamark:production_area:category
          2     seamark:recommended_track:category
          2     seamark:recommended_track:traffic_flow
          2     seamark:rock:depth
          2     seamark:signal_station_traffic:category
          2     seamark:source
          2     seamark:topmark:colour_pattern
          2     service:bicycle:bicycle_sales_and_service
          2     service:bicycle:paintjob
          2     service:computer
          2     service:electricity
          2     service:scan
          2     service:vehicle:audio
          2     service:vehicle:bodywork
          2     service:vehicle:bus_repair
          2     service:vehicle:engine
          2     service:vehicle:glass_repair
          2     service:vehicle:maintenance
          2     service:vehicle:mecanica_general
          2     service:vehicle:motorcycle
          2     service:vehicle:motorcycle_customization
          2     service:vehicle:oil_filter_change
          2     service:vehicle:parts
          2     service:vehicle:plating
          2     service:vehicle:polishing
          2     service:vehicle:radiators
          2     service:vehicle:reboque
          2     service:vehicle:revenda_de_autom├│veis
          2     service:vehicle:tires
          2     service:vehicle:towing
          2     service:vehicle:wheel_alignment
          2     service:vehicle:wheel_balancing
          2     service:vehicle:window_tinting
          2     service_hours
          2     services
          2     shape
          2     shape_area
          2     shopping
          2     short_name:bg
          2     short_name:cnr
          2     short_name:cy
          2     short_name:et
          2     short_name:hr
          2     short_name:it
          2     short_name:lt
          2     short_name:lv
          2     short_name:mi
          2     short_name:tr
          2     short_name_1
          2     sidewalk:both:wheelchair
          2     sidewalk:left:segregated
          2     signal
          2     sir
          2     skatepark:halfpipe
          2     sleeping_car
          2     snack
          2     so
          2     social_centre:for
          2     socket:type2_combo:output
          2     source:4wd_only
          2     source:address
          2     source:brand
          2     source:email
          2     source:hgv:mapillary
          2     source:iata
          2     source:landuse
          2     source:maxweight:hgv:conditional
          2     source:name:pt
          2     source:name:url
          2     source:nat_name
          2     source:network
          2     source:note:name
          2     source:route
          2     source:service
          2     source:species
          2     source:steam
          2     source:waterway
          2     source_ref:url
          2     source_ref:url:moovit
          2     spur
          2     stone
          2     stores
          2     stream:type
          2     surface:proposed
          2     survey_point:datum_aligned
          2     swimwear
          2     table
          2     table_tennis:tables
          2     tactile_writing:braille:lg
          2     tank_trap
          2     taxi_type
          2     taxon:species
          2     tenant
          2     tertiary
          2     ticket
          2     tiger:county
          2     tobacco
          2     toilet
          2     toll:backward
          2     tour
          2     tower:type_1
          2     towpath
          2     tractor:parts
          2     tractor:repair
          2     trail
          2     transit:forward
          2     tunnel:wikidata
          2     turn:lanes:both_ways
          2     tyres
          2     uf
          2     used
          2     valid_from
          2     vehicle:forward
          2     viewpoint
          2     voltage-low
          2     walk
          2     was:building
          2     was:building:levels
          2     was:email
          2     was:end_date
          2     was:historic
          2     was:industrial
          2     was:phone
          2     was:place
          2     water_characteristic
          2     water_slide
          2     waterfall
          2     waterfront
          2     weather:hygrometer
          2     weather:wind_vane
          2     website:en
          2     website:source
          2     weight
          2     wheelchair:step_height
          2     whitewater
          2     width:lanes:backward
          2     width:lanes:forward
          2     wikipedia:fr
          2     xmas:note
          2     xmas:url
          2     year_built
          2     zona_de_fiscalizacao
          1     3dr:height1
          1     3dr:height2
          1     3dr:length2
          1     AREA DO PROJETO
          1     Aquino
          1     Arte Grafite
          1     Association
          1     Atual
          1     A├ºa├¡
          1     Boteco
          1     CHAVE
          1     COD
          1     COD_LOG
          1     Carnaval
          1     DIST_DESC
          1     DPTO
          1     DPTO_DESC
          1     FISK
          1     F├üBRICA DE M├ôVEIS
          1     GNS:dsg_code
          1     GNS:dsg_string
          1     Grafite
          1     Gym
          1     ID1
          1     IGA:NOMECARTA
          1     IJSN:zona
          1     IPP:ORGAO4
          1     Igreja
          1     Industry
          1     Inmetro
          1     LAYER
          1     LONG
          1     LSI
          1     Lavajato
          1     Lindsay
          1     Magnus Diammond
          1     Market
          1     McCafe
          1     M├│veis
          1     NM_BAIRRO
          1     NOVOCEP
          1     PDF_layer
          1     PDF_lineColor
          1     PDF_nr
          1     PK_COD_ENTIDADE
          1     Pamonharia Cunha
          1     Ponte Rio Escondido
          1     Posto de combust├¡vel
          1     Rede
          1     SERVICE
          1     Schalansky_ref
          1     Sede
          1     Source_T
          1     Status
          1     TMC:cid_58:tabcd_1:Class
          1     TMC:cid_58:tabcd_1:LCLversion
          1     TMC:cid_58:tabcd_1:LocationCode
          1     TOPONYME
          1     USO
          1     abandoned:alt_name
          1     abandoned:ford
          1     abandoned:hotel
          1     abandoned:industrial
          1     abandoned:man_made
          1     abandoned:owner
          1     abandoned:resource
          1     abandoned:short_name
          1     abandoned:shoulder
          1     access:emergency
          1     access_1
          1     acronym
          1     actuator
          1     addr2:street
          1     addr:areatype
          1     addr:beach
          1     addr:borough
          1     addr:building
          1     addr:city:pt
          1     addr:division
          1     addr:house
          1     addr:housenumber1
          1     addr:housenumber:en
          1     addr:housenumber_2
          1     addr:km
          1     addr:lot
          1     addr:parentstreet
          1     addr:pluscode:local
          1     addr:pobox
          1     addr:postbox
          1     addr:postcode_1
          1     addr:region
          1     addr:street2
          1     addr:street:en
          1     addr:street:source
          1     addr:tipoarea
          1     addr:type
          1     addr:typearea
          1     addr:warehouse
          1     aerialway:capacity
          1     aerodrome_marking
          1     aeroway:light
          1     ag├¬ncia
          1     airconditioned
          1     airmodel
          1     alley
          1     alt_addr:housenumber
          1     alt_addr:street
          1     alt_cep
          1     alt_name:2
          1     alt_name:az
          1     alt_name:be-tarask
          1     alt_name:bo
          1     alt_name:ckb
          1     alt_name:co
          1     alt_name:csb
          1     alt_name:cy
          1     alt_name:da
          1     alt_name:eu
          1     alt_name:fi
          1     alt_name:fo
          1     alt_name:hr
          1     alt_name:ie
          1     alt_name:ja
          1     alt_name:kn
          1     alt_name:ko
          1     alt_name:ku
          1     alt_name:lt
          1     alt_name:mi
          1     alt_name:mk
          1     alt_name:nah
          1     alt_name:nan
          1     alt_name:new
          1     alt_name:no
          1     alt_name:pl
          1     alt_name:pms
          1     alt_name:ps
          1     alt_name:qu
          1     alt_name:roa-rup
          1     alt_name:ru
          1     alt_name:rw
          1     alt_name:sco
          1     alt_name:sh
          1     alt_name:sk
          1     alt_name:source
          1     alt_name:sq
          1     alt_name:su
          1     alt_name:sv
          1     alt_name:tr
          1     alt_name:wo
          1     alt_name:zh
          1     alt_name:zh-Hans
          1     alt_name:zh-Hant
          1     alt_name_5
          1     alt_network
          1     amenities
          1     amenity:disused
          1     amenity:fast_food
          1     amenity:restaurant
          1     amenity_4
          1     amenity_5
          1     amsl
          1     animal_shelter:purpose
          1     anno
          1     anthropogenic
          1     appliance:repair
          1     arcade
          1     are
          1     area_1
          1     area_ha
          1     artist
          1     artist:wikipedia
          1     artwork
          1     assembly_point:fire
          1     assisted_trail
          1     associatedStreet
          1     atv:sales
          1     authentication:biometric
          1     authentication:none
          1     auto
          1     autor
          1     availability
          1     barechested
          1     barrier:type
          1     bbq
          1     beauty_1
          1     bell:service_times
          1     bic
          1     bicycle:repair
          1     bicycle:sales
          1     bicycle:tyres
          1     biergarten
          1     biodiesel
          1     biome:type
          1     birds_nest
          1     boardwalk
          1     boat:rental
          1     boat:sales
          1     boe
          1     bonfire
          1     bookswap
          1     boundary:type
          1     boundary_type
          1     boundary_type:FR
          1     bridge:old_name
          1     bridge_1
          1     bridge_ref
          1     buiding
          1     building:architect
          1     building:cladding
          1     building:eaves:height
          1     building:end_date
          1     building:entrance
          1     building:facade:material
          1     building:fireproof
          1     building:floor_area
          1     building:frame:material
          1     building:layers
          1     building:level1
          1     building:levels:roof
          1     building:material_1
          1     building:max_levels
          1     building:note
          1     building:repeat_on
          1     building:state_of_the_house
          1     building:structure
          1     building:type
          1     building:use:office
          1     building:walls
          1     building:year_built
          1     building_name
          1     building_supplies:rental
          1     building_type
          1     buiy
          1     buoy:colour
          1     busbar
          1     bush
          1     business
          1     busway:centre
          1     cadeirante
          1     camera:repair
          1     camp
          1     camp_pitch:type
          1     candy
          1     canteen
          1     capacity:car
          1     capacity:motorcycles
          1     capacity:seats
          1     capital_ISO3166-1
          1     car:brakes
          1     car:injection
          1     car:oil_change
          1     car:sales
          1     car:type
          1     car:tyres
          1     cart
          1     cash_withdrawal:purchase_required
          1     catastro:ref
          1     cats
          1     cce
          1     ceck_date:currency:XBT
          1     census
          1     center
          1     cep
          1     cep:impar
          1     cep:par
          1     chalet
          1     channel
          1     chapel
          1     charge:conditional
          1     charging_station
          1     charity
          1     check:date
          1     check_date:currency
          1     check_date:footway:surface
          1     check_date:internet_access
          1     check_date:sidewalk:surface
          1     check_date:smoking
          1     check_date:traffic_signals:vibration
          1     cheese
          1     chences
          1     cheque_in
          1     cicloamigavel:agua
          1     cicloamigavel:banheiro
          1     cicloamigavel:tomada
          1     cidades
          1     cinema:IMAX
          1     class:bicycle:touring
          1     climbing:ice
          1     climbing:mixed
          1     climbing:pitches
          1     climbing:quality
          1     closed:amenity
          1     closed:shop
          1     clothes:children
          1     clothes:for
          1     club_1
          1     cmt
          1     coastline
          1     cocktails
          1     collector
          1     com
          1     communication:2G
          1     communication:2g
          1     communication:Algar
          1     communication:CTBC
          1     communication:claro_br
          1     communication:edge
          1     communication:hamradio
          1     communication:mobile_phone_1
          1     communication:nr
          1     communication:phone
          1     communication:provider
          1     communication:telecom
          1     communication:transmission
          1     communications_transponder:frequency
          1     communications_transponder:modulation
          1     communications_transponder:service
          1     community:for
          1     compensator
          1     compounding
          1     compressed_air:fee
          1     compressed_air:opening_hours
          1     compressed_air:valves
          1     condominium
          1     construction:barrier
          1     construction:healthcare
          1     construction:height
          1     construction:leisure
          1     construction:maxspeed
          1     construction:oneway
          1     construction:residential
          1     construction:shop
          1     contact
          1     contact:blog
          1     contact:blogspot
          1     contact:email_1
          1     contact:emergency_phone
          1     contact:issu
          1     contact:mail
          1     contact:messenger
          1     contact:nostr
          1     contact:phone:emergency
          1     contact:phone_2
          1     contact:snapchat
          1     contact:soundcloud
          1     contact:threads
          1     contact:vhf
          1     continue
          1     convenience
          1     cooking
          1     coreogthere
          1     coreto
          1     corrego
          1     counselling_type:nutrition
          1     couples
          1     court
          1     courts
          1     cow
          1     crane:maxload
          1     criminality_tax
          1     crosswalk
          1     cr├ís
          1     cult
          1     cultura
          1     cultural
          1     culture
          1     currency:ADA
          1     currency:BOB
          1     currency:BYN
          1     currency:BYR
          1     currency:BsF
          1     currency:CHF
          1     currency:CZK
          1     currency:DAI
          1     currency:DZA
          1     currency:DePix
          1     currency:HUF
          1     currency:INR
          1     currency:LBTC
          1     currency:MATIC
          1     currency:MXN
          1     currency:NGN
          1     currency:PLN
          1     currency:RSD
          1     currency:RUB
          1     currency:THB
          1     currency:TUSD
          1     currency:UAH
          1     currency:USDP
          1     currency:XNO
          1     currency:rial
          1     cut
          1     cycleway:buffer
          1     cycleway:colour
          1     cycleway:left:buffer
          1     cycleway:right:bicycle
          1     cycleway:right:buffer
          1     cycleway:right:separation:left
          1     cycleway:right:surface:colour
          1     cycleway_1
          1     dam
          1     dance
          1     dataset
          1     datum
          1     decorative
          1     def:highway=living_street;maxspeed
          1     def:highway=motorway;access:bicycle
          1     def:highway=motorway;access:foot
          1     def:highway=motorway;access:moped
          1     def:highway=motorway;lanes
          1     def:highway=motorway;maxspeed
          1     def:highway=motorway;maxspeed:rain
          1     def:highway=motorway;oneway
          1     def:highway=primary|highway=secondary|highway=tertiary;maxs
          1     def:highway=residential;maxspeed
          1     def:highway=trunk;access:bicycle
          1     def:highway=trunk;access:foot
          1     def:highway=trunk;access:moped
          1     def:highway=trunk;maxspeed
          1     def:rural,unit=kmh
          1     def:urban,unit=kmh
          1     deleted:waterway
          1     delivery:covid19
          1     demolished:barrier
          1     demolished:building:material
          1     demolished:community_centre
          1     demolished:leisure
          1     demolished:operator
          1     demolished:public_transport
          1     demolished:roof:orientation
          1     demolished_building
          1     departures_board:speech_output
          1     deprecated:amenity
          1     des
          1     description2
          1     description:ru
          1     desde
          1     destination:arrow
          1     destination:colour:arrow:lanes
          1     destination:colour:text:lanes
          1     destination:distance
          1     destination:lang:de
          1     destination:lang:es:lanes
          1     destination:lang:pt
          1     destination:street:backward
          1     destination:symbol:forward
          1     destination_1
          1     destroyed:amenity
          1     destroyed:brand
          1     destroyed:bridge:material
          1     destroyed:date
          1     destroyed:historic
          1     destroyed:man_made
          1     destroyed:oneway
          1     destroyed:power
          1     destroyed:tourism
          1     destroyed:waterway
          1     device
          1     dgpc:inscription_date
          1     diaper
          1     diet
          1     diet:Saudavel
          1     diet:churrasco_raiz
          1     diet:dairy
          1     diet:friture
          1     diet:horta
          1     diet:junkfood
          1     diet:lacto-ovo_vegetarian
          1     diet:macrobiotic
          1     diet:nut_free
          1     diet:pastel
          1     diet:prot├®ica
          1     diet:regular
          1     diet:slow_food
          1     diet:smoke-dried
          1     diet:soy_free
          1     digital
          1     diplomatic:services:Notarial_Services
          1     diplomatic:services:passport
          1     direction:forward
          1     direction_east
          1     direction_south
          1     dirt
          1     disabled:aeroway
          1     disabled_spaces
          1     disc_golf
          1     dismount
          1     display_name
          1     dispute
          1     disputed
          1     disused:access
          1     disused:capacity
          1     disused:clothes
          1     disused:cycleway:right
          1     disused:cycleway:right:oneway
          1     disused:delivery
          1     disused:door
          1     disused:footway
          1     disused:garden:type
          1     disused:generator:source
          1     disused:information
          1     disused:military
          1     disused:name_2
          1     disused:natural
          1     disused:official_name
          1     disused:opening_hours
          1     disused:outdoor_seating
          1     disused:social_facility
          1     disused:source:name
          1     disused:train_station
          1     disused:watermill
          1     disused:website
          1     division_code
          1     dogs
          1     donation
          1     donation:facility
          1     dragon
          1     drink:alcohol
          1     drink:brewery
          1     drink:espresso
          1     drink:milk
          1     drink:raw_milk
          1     drinking_water:official
          1     drinking_water:refill
          1     drinks
          1     drone
          1     dune
          1     duration:backward
          1     duration:forward
          1     ele:accuracy
          1     ele:local
          1     electricity
          1     electronics
          1     email2
          1     email:eventos
          1     embedded_rails
          1     emergency:lanes
          1     employee
          1     end_date:building
          1     engineer:type
          1     engineering
          1     entrance_model
          1     ephemeral
          1     erotic
          1     escalator_dir
          1     est_avance
          1     estuary
          1     ethnicity
          1     etymology:wikidata
          1     euro_const
          1     exhibit
          1     extension_phone
          1     extrude
          1     fabrica
          1     fair
          1     fazenda
          1     fee:amount
          1     fee:price
          1     fence:height
          1     festival
          1     filial
          1     filming
          1     finish_date
          1     fire
          1     fire_hydrant:couplings
          1     fire_rank
          1     fire_station:type
          1     fire_station:type:FR
          1     fitness_station:horizontal_bar
          1     fitness_station:parallel_bars
          1     fixeme
          1     fixme:br
          1     fixme:en
          1     fixme:pt
          1     fixme:tourism
          1     fixme:website
          1     flagpole
          1     flagpole:construction
          1     floor
          1     foliage_persistence
          1     football
          1     footway:right:oneway
          1     for
          1     former:landuse
          1     founder:wikipedia
          1     franchise
          1     free_flying:site_orientation
          1     fuel:Arla
          1     fuel:Diesel_Tech
          1     fuel:Gasoline
          1     fuel:Grid
          1     fuel:Shell_V-Power
          1     fuel:adblue:canister
          1     fuel:arla
          1     fuel:coal
          1     fuel:diesel_1
          1     fuel:diesel_S00
          1     fuel:diesel_comum
          1     fuel:diesel_evolux
          1     fuel:energy
          1     fuel:engine_oil
          1     fuel:gasohol_95
          1     fuel:gasolina_C
          1     fuel:gasolina_C_premium
          1     fuel:gasoline_93
          1     fuel:g├ís
          1     fuel:kerosene
          1     fuel:mdo
          1     fuel:mgo
          1     fuel:octane_102
          1     fuel:octane_89
          1     full_service
          1     functional
          1     funeral_hall
          1     futebol:teaching
          1     game
          1     gantry
          1     garagem
          1     gas
          1     gated
          1     gender
          1     generator:output:cold_water
          1     generator:output:steam
          1     generator:solar:modules
          1     geological:outcrop
          1     gid
          1     gift
          1     glasses
          1     gnis:county_id
          1     gnis:ftype
          1     golf:designer
          1     golf:stroke_index
          1     grade
          1     green_roof
          1     ground
          1     guide
          1     habitat
          1     hackerspace
          1     harbor
          1     harbour:boat_hoist
          1     harbour:electricity
          1     harbour:fuel_station
          1     harbour:nautical_club
          1     harbour:pump-out
          1     harbour:refuse_bin
          1     harbour:showers
          1     harbour:slipway
          1     harbour:toilets
          1     harbour:visitor_berth
          1     harbour:water_tap
          1     hashtags
          1     hazard:backward
          1     health_facility:dentistry
          1     health_food
          1     health_service:counselling
          1     health_specialty:gynaecology
          1     health_specialty:nephrology
          1     health_specialty:pediatrics
          1     health_specialty:physiatry
          1     health_specialty:radiology
          1     health_specialty:yoga
          1     heat
          1     heated
          1     hectares
          1     height_1
          1     herb
          1     hgv:backward
          1     high_pressure_washer
          1     highway:1998-2002
          1     highway:2000-2008
          1     highway:2002-2008
          1     highway:2002-2009
          1     highway:2002-2010
          1     highway:2003-2011
          1     highway:2003-2013
          1     highway:2004-2005
          1     highway:2004-2007
          1     highway:2006-2012
          1     highway:2007-2009
          1     highway:2007-2013
          1     highway:2008-2012
          1     highway:abandoned
          1     highway:lane
          1     highway:xxxx-xxxx
          1     highway_2
          1     highway_3
          1     historic:admin_level
          1     historic:boundary
          1     historic:highway
          1     historic:landuse
          1     historic:place
          1     historic:power
          1     historic:power:plant:source
          1     historic:service
          1     historic:website
          1     home
          1     homeopatic
          1     horario de funcionamento
          1     horarios
          1     hou
          1     ibge:id
          1     id:clearmedia
          1     id_areas_e
          1     id_clearchannel
          1     identity
          1     ignore
          1     image_direction
          1     import
          1     inauguration_date
          1     incomplete
          1     incorrect:lanes
          1     incorrect:man_made
          1     incorrect:maxspeed
          1     incorrect:tower:type
          1     ine:municipio
          1     inepe
          1     inner
          1     inscription:backward
          1     inscription:forward
          1     inscription_0
          1     inscription_1
          1     int
          1     internet_access:access
          1     iphan:ref
          1     is_in:island
          1     is_in:subregion
          1     is_in:village
          1     is_in:waterway
          1     is_sidepath
          1     isced:2011:level
          1     isolation
          1     jacuzzi
          1     jetski:rental
          1     jetski:sales
          1     juscelino
          1     kayak_rental
          1     key_cutter
          1     kindergarten
          1     kissing_gate
          1     laborat├│rio
          1     lagoon
          1     lamp_count
          1     land
          1     land_property
          1     landfill:type:FR
          1     landform
          1     landmark
          1     landscape
          1     landuse_3
          1     lanes:bus:backward:conditional
          1     lanes:emergency:backward
          1     lanes:taxi
          1     language:ar
          1     language:be
          1     language:br
          1     language:el
          1     language:hin
          1     language:ko
          1     language:nl
          1     language:pl
          1     language:pt-BR
          1     language:ro
          1     language:ru
          1     language:tok
          1     language:uk
          1     language:vi
          1     lay
          1     lengths:right
          1     lg_prep
          1     lg_titulo
          1     lgbtq:men
          1     liaison
          1     light_source
          1     loc_name:es
          1     loc_name:it
          1     loc_name:pt
          1     locality
          1     locker
          1     locksmith
          1     locomotive:start_date
          1     logo
          1     long_jump
          1     lookout
          1     lot
          1     loteamento
          1     love_locks
          1     lte:eNB
          1     luggage_locker
          1     lwn_ref
          1     main
          1     maintenance
          1     man_made:material
          1     man_made_1
          1     manufacturer:furniture
          1     marine
          1     marsh water
          1     mata atlantica
          1     material:de
          1     maxspeed:bus:forward
          1     maxspeed:school_time
          1     maxweight:bus
          1     maxweight:hgv
          1     maybe:rapids
          1     maze
          1     measurement
          1     medical_supply
          1     medicine
          1     mental_health
          1     metal
          1     micropub
          1     military_1
          1     min_radius
          1     mine
          1     mobile_phone
          1     mock
          1     model_aerodrome:combustion
          1     model_aerodrome:electric
          1     model_aerodrome:heli
          1     model_aerodrome:turbine
          1     monitoring:air
          1     monitoring:air_pressure
          1     monitoring:galileo
          1     monitoring:gas
          1     monitoring:geodetic
          1     monitoring:ombp
          1     monitoring:particulate_matter
          1     monitoring:rastreamento_veicular
          1     monitoring:sky_brightness
          1     monitoring:tide_gauge
          1     monitoring:wind_direction
          1     motorcycle:brakes
          1     motorcycle:lanes:backward
          1     motorcycle:lanes:forward
          1     motorcycle:safety_inspection
          1     motorcycle:training
          1     motorcycle_friendly
          1     moved
          1     movimento
          1     mp_type
          1     mtg
          1     mtg:type
          1     museum:start_date
          1     music_genre
          1     musical_instrument
          1     musical_instrument:repair
          1     nae
          1     name:-1871
          1     name:-2017
          1     name:az-Arab
          1     name:az-Cyrl
          1     name:backward
          1     name:bpv
          1     name:cb
          1     name:comment
          1     name:de_DE
          1     name:forward
          1     name:gaceria
          1     name:glk
          1     name:got
          1     name:grc
          1     name:guc
          1     name:hil
          1     name:iu
          1     name:ja_kana
          1     name:ja_rm
          1     name:ko-Latn
          1     name:map-bms
          1     name:mo
          1     name:pt-PT
          1     name:pth
          1     name:ru:word_stress
          1     name:sje
          1     name:tca
          1     name:ttc
          1     name:wikidata
          1     name:zh-Latn-pinyin
          1     name_4
          1     nat_name:en
          1     nat_name:es
          1     nautical_club
          1     ne
          1     nem
          1     neme
          1     network:operator
          1     network_4
          1     newspaper
          1     nickname:en
          1     nickname:fi
          1     nickname:pl
          1     nivel
          1     nm_bairro
          1     nma
          1     no:railway
          1     noe
          1     noise
          1     norma_creacion
          1     norte
          1     not:access
          1     not:addr:city
          1     not:addr:postcode
          1     not:bicycle
          1     not:inscription
          1     not:is_in:city
          1     not:place
          1     not:public_transport
          1     not:tracktype
          1     not_served_by
          1     not_visible
          1     note2
          1     note:2
          1     note:ISO3166-1
          1     note:crossing
          1     note:description
          1     note:ele
          1     note:geometry
          1     note:hazard
          1     note:ja
          1     note:offset
          1     note:oneway
          1     note:pl
          1     note:relevance
          1     note:start_date
          1     note:website
          1     note:wikidata
          1     note_2
          1     numero
          1     nutrition_supplements
          1     oaci
          1     obelisk:height
          1     obelisk:material
          1     object
          1     observation
          1     obstacle:wheelchair
          1     official
          1     official_name:ab
          1     official_name:arc
          1     official_name:as
          1     official_name:av
          1     official_name:bar
          1     official_name:bat-smg
          1     official_name:ceb
          1     official_name:co
          1     official_name:cu
          1     official_name:diq
          1     official_name:dty
          1     official_name:eml
          1     official_name:frp
          1     official_name:fur
          1     official_name:ga
          1     official_name:gag
          1     official_name:gcf
          1     official_name:guc
          1     official_name:haw
          1     official_name:hif
          1     official_name:jbo
          1     official_name:jv
          1     official_name:km
          1     official_name:kw
          1     official_name:lij
          1     official_name:ln
          1     official_name:lrc
          1     official_name:min
          1     official_name:mo
          1     official_name:mr
          1     official_name:mrj
          1     official_name:myv
          1     official_name:new
          1     official_name:nn
          1     official_name:nov
          1     official_name:pag
          1     official_name:pcd
          1     official_name:pdc
          1     official_name:pih
          1     official_name:roa-rup
          1     official_name:sc
          1     official_name:scn
          1     official_name:sd
          1     official_name:so
          1     official_name:srn
          1     official_name:stq
          1     official_name:szl
          1     official_name:tg
          1     official_name:tyv
          1     official_name:udm
          1     official_name:vo
          1     official_name:za
          1     official_name:zea
          1     old:amenity
          1     old:oneway
          1     old_addr:country
          1     old_addr:housenumber2
          1     old_addr:suburb
          1     old_alt_name
          1     old_building
          1     old_capacity
          1     old_cep
          1     old_contact:website
          1     old_email
          1     old_housenumber
          1     old_name:-1825
          1     old_name:1825-01-29--1825-08-13
          1     old_name:1825-08-13--1826-11-19
          1     old_name:1825-1826
          1     old_name:1826-11-29--1837-05-01
          1     old_name:1830-1916
          1     old_name:1837-05-01--1839-10-26
          1     old_name:1839-10-26--2009-03-18
          1     old_name:1871-1921
          1     old_name:1906
          1     old_name:1916-1945
          1     old_name:1940
          1     old_name:1969-1996
          1     old_name:2016-2019
          1     old_name:am
          1     old_name:ca
          1     old_name:de
          1     old_name:ko
          1     old_name:si
          1     old_name:zh
          1     old_network
          1     old_no
          1     old_note
          1     old_operator:wikidata
          1     old_operator:wikipedia
          1     old_railway_operator
          1     old_ref:itu
          1     old_source
          1     old_source:name
          1     old_sport
          1     old_start_date
          1     old_website_1
          1     ona
          1     oneway:2015-09-29--
          1     oneway:2015-10-22--
          1     oneway:2015-10-23--
          1     oneway:2015-10-27--
          1     oneway:motorcycle
          1     open_air
          1     opening
          1     opening_days
          1     opening_hours:secretary
          1     opening_hours:self_service
          1     operator:addr
          1     operator:short_name
          1     operator:wikipedia:ru
          1     operator_2
          1     orchard
          1     osm_note
          1     osmc:ref
          1     other_use
          1     owner:type
          1     pak
          1     panoramax:3
          1     parasol
          1     park_type
          1     parking:both:authentication:ticket
          1     parking:both:maxstay:conditional
          1     parking:both:restriction:hgv
          1     parking:both:restriction:reason
          1     parking:both:surface
          1     parking:condition
          1     parking:condition:either_side_only
          1     parking:condition:left:hgv
          1     parking:condition:maxstay
          1     parking:condition:right:capacity:disabled
          1     parking:customer
          1     parking:lane:left:condition
          1     parking:lanes
          1     parking:left:authentication:ticket
          1     parking:left:hgv
          1     parking:left:motorcar
          1     parking:markings
          1     parking:placement
          1     parking:right:authentication:ticket
          1     parking:right:restriction:police
          1     parking:right:surface
          1     parking:right:tourist_bus
          1     part_time_beds
          1     pastel
          1     pasture
          1     patron_saint:de
          1     patron_saint:en
          1     patron_saint:wikidata
          1     pavement
          1     pavimentac
          1     pay_tv
          1     payment: app-bike ita├║
          1     payment:A_vista
          1     payment:Bilhete_Unico_Metropolitano
          1     payment:Boleto_banc├írio
          1     payment:Cart├úo_de_Passe
          1     payment:Cursos_Gratuitos
          1     payment:NFC_mobile_payments
          1     payment:Tigo_Money
          1     payment:Vales_electronicos
          1     payment:ada_cardano
          1     payment:alelo_alimenta├º├úo
          1     payment:alipay
          1     payment:ame_digital
          1     payment:apag
          1     payment:atacado
          1     payment:banco_do_brasil
          1     payment:banese
          1     payment:bank_deposits
          1     payment:banrisul
          1     payment:bcash
          1     payment:be_blue
          1     payment:ben_alimenta├º├úo
          1     payment:bilhete-unico
          1     payment:binance
          1     payment:blink
          1     payment:bradesco
          1     payment:cabal
          1     payment:caixatem
          1     payment:card_lokal
          1     payment:cardano
          1     payment:cart├úo_atacad├úo
          1     payment:cart├úo_carrefour
          1     payment:cielo
          1     payment:credit_account
          1     payment:currency
          1     payment:diners
          1     payment:dotz
          1     payment:electronic_transfer
          1     payment:ep_easycard
          1     payment:ep_ipass
          1     payment:financing
          1     payment:food_card
          1     payment:food_stamps
          1     payment:fortbrasil
          1     payment:invoice
          1     payment:ita├║
          1     payment:link_de_pagamento
          1     payment:martexcoin
          1     payment:mastercard_electronic
          1     payment:mumbuca
          1     payment:on-chain
          1     payment:others
          1     payment:parcelado
          1     payment:payback
          1     payment:polishop
          1     payment:qr_pay
          1     payment:santander
          1     payment:sats
          1     payment:sem_custo
          1     payment:senff
          1     payment:sumup
          1     payment:tauste_pay
          1     payment:tether
          1     payment:tetherus
          1     payment:token
          1     payment:v_pay
          1     payment:varejo
          1     payment:via_facil
          1     payment:vuoncard
          1     payment:wallet
          1     payment:website:bitcoin
          1     payment:wechat
          1     payment:whatsapp
          1     paymente:coinos
          1     pedagogy
          1     pedalboat_rental
          1     people
          1     perennial
          1     perfumery
          1     permanent
          1     per├¡odo
          1     pet_hotel
          1     phone:1
          1     phone:AR
          1     phone:Claro
          1     phone:Tim
          1     picnic
          1     picnic_shelter
          1     picture
          1     pilgrimage
          1     pitch
          1     place_1
          1     placement:backward
          1     planned:landuse
          1     plant:output:compressed_air
          1     plant:output:hot_water
          1     plant:output:steam
          1     plant_community:it
          1     planted
          1     planted_by
          1     planted_date
          1     playground:roundabout
          1     playground:seesaw
          1     playground:slide
          1     playground:structure
          1     playground:swing
          1     playground_1
          1     playground_2
          1     plumber
          1     pmmcr:id
          1     police:type
          1     polling_station:zone
          1     popular name
          1     population:source
          1     porn_movies
          1     post:city
          1     post:postcode
          1     post_box
          1     post_box:type
          1     post_office:letter_from
          1     post_office:parcel_from
          1     post_office:parcel_to
          1     postal_code:source
          1     poultry
          1     power:disused
          1     power_source
          1     primary_link
          1     processing
          1     procted_class
          1     prog
          1     project
          1     proposed:aerialway:capacity
          1     proposed:aerialway:duration
          1     proposed:aerialway:occupancy
          1     proposed:bridge
          1     proposed:office
          1     proposed:oneway
          1     protection_aim
          1     protection_title:en
          1     provided_for:child
          1     psy
          1     public_space
          1     publishing
          1     pump_mechanism
          1     radiator:repair
          1     radio_transponder:sequence
          1     radio_transponder:signal_frequency
          1     radius
          1     railway:pzb
          1     railway_1
          1     rastreamento
          1     razed:amenity
          1     razed:end_date
          1     razed:name
          1     razed:operator
          1     razed:start_date
          1     razed:usage
          1     re
          1     reception_desk
          1     recommendation
          1     recreation center
          1     recycling:Blister_de_rem├®dio
          1     recycling:Blisters_de_rem├®dios
          1     recycling:Cosm├®ticos_vencidos
          1     recycling:Garrafas_de_iogurte
          1     recycling:Qualquer_tampa_de_pl├ístico
          1     recycling:bottles
          1     recycling:caixa_de_ovos
          1     recycling:cardbox
          1     recycling:ceramic_toilets
          1     recycling:cobre
          1     recycling:compost
          1     recycling:compressor
          1     recycling:copper
          1     recycling:food_waste
          1     recycling:household_trash
          1     recycling:little_can
          1     recycling:medicines
          1     recycling:mobile_phone_chargers
          1     recycling:motor
          1     recycling:needles
          1     recycling:organic
          1     recycling:plastic_bags
          1     recycling:plastic_bottle_caps
          1     recycling:pmd
          1     recycling:polyester
          1     recycling:printer_inkjet_cartridges
          1     recycling:printer_toner_cartridges
          1     recycling:profile
          1     recycling:zinc
          1     red_turn:right:bicycle
          1     redwood_city_ca:bld_gid
          1     ref:ANA
          1     ref:AR:CAMMESA
          1     ref:FR:ANFR
          1     ref:FR:FINESS
          1     ref:FR:INPN
          1     ref:FR:SDIS:GRPT
          1     ref:FR:SINOE
          1     ref:SNIIC
          1     ref:UY:minterior:comisaria_id
          1     ref:aatl
          1     ref:agrijgua
          1     ref:arlhs
          1     ref:b3
          1     ref:branch
          1     ref:brand
          1     ref:cec
          1     ref:color
          1     ref:condephaat
          1     ref:cro
          1     ref:inmet
          1     ref:jgs
          1     ref:linz:building_id
          1     ref:lot:irreg
          1     ref:note
          1     ref:nuts
          1     ref:nuts:1
          1     ref:nuts:2
          1     ref:nuts:3
          1     ref:oep:ted
          1     ref:old
          1     ref:ourairports
          1     ref:secult
          1     ref:source
          1     ref:spepm
          1     reflective:colour
          1     rehabilitation:speciality
          1     related_law:country
          1     religion:it
          1     religion:wikidata
          1     remark
          1     removed:amenity
          1     removed:covered
          1     removed:historic
          1     removed:landuse
          1     removed:note
          1     removed:operator
          1     removed:payment:telephone_cards
          1     rer
          1     reservation:phone
          1     restricted
          1     restriction:bus:conditional
          1     restriction:psv
          1     restriction_no_left_turn
          1     ret
          1     reusable_packaging:offer
          1     right:4
          1     right:region
          1     right:state
          1     risco
          1     roller_coaster:track
          1     roller_skates:rental
          1     roof:wall
          1     rooftop
          1     ruins:amenity
          1     ruins:denomination
          1     ruins:highway
          1     ruins:religion
          1     ruins:surface
          1     rungs
          1     salon
          1     sat
          1     school:ML:academie
          1     school_type
          1     science
          1     scuba_diving:courses
          1     scuba_diving:dangers
          1     scuba_diving:dangers:obstacles
          1     scuba_diving:depth
          1     scuba_diving:entry
          1     scuba_diving:type
          1     scuba_diving:type:training
          1     sculpture
          1     seafood:sales
          1     seal
          1     seamark:beacon_cardinal:shape
          1     seamark:beacon_isolated_danger:shape
          1     seamark:beacon_lateral:colour_pattern
          1     seamark:beacon_special_purpose:colour_pattern
          1     seamark:berth:goods
          1     seamark:bridge:clearance_height_closed
          1     seamark:bridge:clearance_height_open
          1     seamark:bridge:clearance_height_safe
          1     seamark:cable_overhead:clearance_height
          1     seamark:construction
          1     seamark:fixme
          1     seamark:harbour:type
          1     seamark:hulk:category
          1     seamark:landmark:colour
          1     seamark:light:2:category
          1     seamark:light:2:sequence
          1     seamark:light:3:category
          1     seamark:light:4:period
          1     seamark:light:5:period
          1     seamark:light:6:character
          1     seamark:light:6:colour
          1     seamark:light:6:height
          1     seamark:light:6:period
          1     seamark:light:6:range
          1     seamark:light:6:sector_end
          1     seamark:light:6:sector_start
          1     seamark:national_name
          1     seamark:oil_barrier:category
          1     seamark:pipeline_submarine:name
          1     seamark:radio_station:channel
          1     seamark:radio_station:group
          1     seamark:recommended_track:orientation
          1     seamark:wall:category
          1     seamark:wreck:depth
          1     seat
          1     secultfor:inscription_date
          1     secultfor:status
          1     secultfor:tombo
          1     sells
          1     sells:haberdashery
          1     service:bicycle:accessories
          1     service:bicycle:bicycle_tours
          1     service:bicycle:bike_tools
          1     service:bicycle:charging
          1     service:branding
          1     service:design
          1     service:graphic_design
          1     service:maketing
          1     service:phone
          1     service:socialmedia
          1     service:tyres
          1     service:vehicle:Reparo_Ba├║s
          1     service:vehicle:Retificadora
          1     service:vehicle:Sale
          1     service:vehicle:air_filter
          1     service:vehicle:alternator
          1     service:vehicle:auto_shielding
          1     service:vehicle:automotive_lighthouse
          1     service:vehicle:automotive_repair
          1     service:vehicle:balanceo
          1     service:vehicle:baterias_heliar
          1     service:vehicle:breakdown
          1     service:vehicle:bus_sales
          1     service:vehicle:cambio
          1     service:vehicle:car_care
          1     service:vehicle:car_design
          1     service:vehicle:car_maintenance
          1     service:vehicle:car_rental
          1     service:vehicle:car_wash
          1     service:vehicle:car_wheel_recuperator
          1     service:vehicle:carwash
          1     service:vehicle:chaveiro
          1     service:vehicle:cleaning
          1     service:vehicle:clutches
          1     service:vehicle:converter
          1     service:vehicle:diesel_center
          1     service:vehicle:engine_diesel
          1     service:vehicle:engine_rebuild
          1     service:vehicle:escapes
          1     service:vehicle:filters
          1     service:vehicle:fuel_conversion
          1     service:vehicle:fuel_modification
          1     service:vehicle:gas
          1     service:vehicle:geometry
          1     service:vehicle:grinding_of_motors
          1     service:vehicle:guincho
          1     service:vehicle:insulfilm
          1     service:vehicle:insurance
          1     service:vehicle:latoaria
          1     service:vehicle:leakage
          1     service:vehicle:leather
          1     service:vehicle:lights
          1     service:vehicle:loja_de_bateria
          1     service:vehicle:lpg
          1     service:vehicle:motor_parts
          1     service:vehicle:muffler_installation
          1     service:vehicle:officina
          1     service:vehicle:old_car_sales
          1     service:vehicle:paint_correction
          1     service:vehicle:particular
          1     service:vehicle:performance
          1     service:vehicle:p├ítio
          1     service:vehicle:radiator
          1     service:vehicle:rear_view_mirror
          1     service:vehicle:remap
          1     service:vehicle:repair_in_meter
          1     service:vehicle:service
          1     service:vehicle:servicio_automotriz
          1     service:vehicle:shield
          1     service:vehicle:shock_absorbers
          1     service:vehicle:small_car_repairs
          1     service:vehicle:som
          1     service:vehicle:speedmeter
          1     service:vehicle:starters
          1     service:vehicle:steering_wheel
          1     service:vehicle:suzuki_mototes
          1     service:vehicle:tracker
          1     service:vehicle:transport
          1     service:vehicle:troca_oleo
          1     service:vehicle:truck_tyres
          1     service:vehicle:tyres_balance
          1     service:vehicle:t├íxi
          1     service:vehicle:vehicle_shielding
          1     service:vehicle:wash
          1     service:vehicle:welding
          1     service:vehicle:wheel_repair
          1     service:wifi
          1     service_times:de
          1     service_times:pt
          1     service_times:url
          1     serving_system:rodizio
          1     sewer
          1     sewing_machine:repair
          1     shape_len
          1     ship_transport
          1     shoes:women
          1     shop:bakery
          1     shop:butcher
          1     shop:computer
          1     shop:confectionery
          1     shop:convenience
          1     shop:name
          1     shop:water
          1     short_name:an
          1     short_name:ar
          1     short_name:ast
          1     short_name:be
          1     short_name:bs
          1     short_name:ca
          1     short_name:co
          1     short_name:da
          1     short_name:el
          1     short_name:eo
          1     short_name:ext
          1     short_name:fi
          1     short_name:frp
          1     short_name:fy
          1     short_name:ga
          1     short_name:gl
          1     short_name:gn
          1     short_name:hu
          1     short_name:ia
          1     short_name:ja
          1     short_name:kbp
          1     short_name:kk
          1     short_name:lb
          1     short_name:li
          1     short_name:mk
          1     short_name:mt
          1     short_name:mwl
          1     short_name:nap
          1     short_name:pa
          1     short_name:pcd
          1     short_name:ro
          1     short_name:sc
          1     short_name:sk
          1     short_name:sl
          1     short_name:sr
          1     short_name:vls
          1     short_name:wa
          1     short_name:zea
          1     short_name:zh
          1     short_name:zh-Hans
          1     short_name:zh-Hant
          1     short_official_name
          1     show
          1     show_house
          1     shower:capacity
          1     shower:fee
          1     shower:unisex
          1     shrubs
          1     sidewalk:left:kerb
          1     sidewalk:left:wheelchair
          1     sidewalk:right:smoothness
          1     sidewalk:right:wheelchair
          1     sidewalk:smoothness
          1     sidewalk:tactile_paving
          1     sidewalk_orientation
          1     signal_box
          1     signed_direction
          1     signpost
          1     singles
          1     site:id
          1     skateboard:rental
          1     sloped_kerb
          1     snowboard
          1     sobrado
          1     social_facility:for_1
          1     socket:chademo
          1     socket:device:USB-A
          1     socket:type2:output
          1     solar_photovoltaic_panel:sales
          1     source1
          1     source:GPS
          1     source:IBGE:CD_ADMINIS
          1     source:ISO3166-1
          1     source:addr:full
          1     source:animal
          1     source:bridge:old_name
          1     source:building:flats
          1     source:building:height
          1     source:charge
          1     source:construction
          1     source:covered
          1     source:cuisine
          1     source:demolished:name
          1     source:euro_const
          1     source:geometry:ref
          1     source:geometry_1
          1     source:healthcare:speciality
          1     source:icao
          1     source:internet_access
          1     source:is_in:district
          1     source:loc
          1     source:maxlength:hgv
          1     source:name:mapillary
          1     source:note:en
          1     source:old_name:1970
          1     source:opening_date
          1     source:population:date
          1     source:ref:INSEE
          1     source:ref:LRP
          1     source:ref:PIR
          1     source:related_law
          1     source:route_master
          1     source:sour
          1     source:sqkm
          1     source:tream
          1     source:tunnel
          1     source:wlan
          1     source_date
          1     source_ref_1
          1     source_url
          1     soure
          1     spare
          1     speed_camera:direction
          1     speed_tax
          1     spices
          1     sport:olympic
          1     sport:paralympic
          1     spring
          1     spring:type
          1     sqkm
          1     stairs:type
          1     standup_paddleboard_rental
          1     start
          1     start_date:university
          1     stationery
          1     stays
          1     stilts
          1     stopping:right
          1     store_1
          1     student:count
          1     subname
          1     sunglasses
          1     supermarky
          1     swimming_lanes
          1     tactile_writing
          1     takeaway:covid19
          1     tank
          1     taxiway
          1     taxon:wikidata
          1     teleatendimento
          1     telecommunication
          1     telescope:usage
          1     television:repair
          1     temporary
          1     temporary:hgv
          1     terminal
          1     theatre:stage
          1     tiger:name_base
          1     tiger:name_direction_prefix
          1     tiger:zip_left
          1     tiger:zip_right
          1     timing
          1     tipo_const
          1     tmp
          1     toilets:changing_table
          1     toilets:fee
          1     toilets:hands_drying
          1     toilets:wheelchair:gender_segregated
          1     toll:foot
          1     tourism_2
          1     tourist_bus:conditional
          1     towel
          1     tower:platforms
          1     town
          1     townhall
          1     toys
          1     track
          1     tractor:sales
          1     traffic
          1     traffic_calming:backward
          1     traffic_calming:forward
          1     traffic_calming:left
          1     traffic_island
          1     traffic_signals:floor_light
          1     traffic_signals:floor_vibration
          1     traffic_signals:turn
          1     traffic_signals:turn:lanes
          1     trainee
          1     transformer:devices
          1     transit
          1     transport:zone
          1     trash
          1     tree:ref
          1     tree_lined
          1     trolley_wire:note
          1     truck:tyres
          1     truck_wash
          1     turn:forward
          1     turn_to_close
          1     typ
          1     type_impact
          1     types
          1     tyres_repair
          1     underground
          1     ursea:cod_estacion
          1     vacuum_cleaner
          1     vatin
          1     vegan
          1     vegetable
          1     vegetation_type
          1     vehicle:wikidata
          1     vehicle:wikipedia
          1     vending:comics
          1     ventilation_shaft
          1     vfr
          1     video:rental
          1     view
          1     village
          1     voltage:comment
          1     war_memorial
          1     warm_water
          1     was:brand
          1     was:contact:mobile
          1     was:covered
          1     was:denomination
          1     was:description
          1     was:healthcare
          1     was:inscription
          1     was:memorial
          1     was:museum
          1     was:natural
          1     was:office
          1     was:official_name
          1     was:payment:telephone_cards
          1     was:product
          1     was:railway
          1     was:residential
          1     was:school
          1     was:sport
          1     was:studio
          1     was:wholesale
          1     waste_basket
          1     waste_disposal
          1     water_well
          1     water_works
          1     waterway:llid
          1     waterway_category:WGF
          1     weather:ceiling_projector
          1     weather:psychrometer
          1     weather:pyranometer
          1     weather:pyrgeometer
          1     weather:pyrheliometer
          1     weather:soil_thermometer
          1     weather:sunshine_recorder
          1     website:1
          1     website:podcast
          1     website:stock
          1     website_2
          1     wedding_reception
          1     weelspeed
          1     weir
          1     western_union
          1     wheelchair:description:de
          1     wheelchair:entrance_width
          1     wikidata:note
          1     wikipedia:de
          1     windings:auto
          1     windings:configuration
          1     windmill:disused
          1     windmill:vanes
          1     window:frame:material
          1     window:shape
          1     window:type
          1     wood:origin
          1     wreck:date_commissioned
          1     wreck:depth
          1     wreck:type
          1     xmas:old_name
          1     xmas:operator
          1     yard
          1     zone:colour
          1     zoning
          1     ├ürea
"""